In [106]:
#50Hz動画完成版?（スケールバー小数対応版）
# === 全フレームを動画化（レイアウトはインプット済みを厳守） ===
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# ---------------- フォント ----------------
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font  = ImageFont.truetype(font_path, 25)
font_I = ImageFont.truetype(font_path_I, 30)

def put_text(img, text, position, font, color=(255,255,255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# ---------------- 共通レイアウト定数（インプット済みを厳守） ----------------
pad = 40
border_thickness = 2
line_color = (255, 255, 255)

INFO = {
    "isop": dict(
        orig_w=601, orig_h=401, xy_w=500, xy_h=300, gap=3,
        scalebar_native=48.84,
        trim_x=98,  # 下枠を右端から短く（isop）
        trim_y=98   # 右枠を下端から短く（isop）
    ),
    "spin": dict(
        orig_w=603, orig_h=603, xy_w=512, xy_h=512, gap=3,
        scalebar_native=46.51,
        trim_x=88,  # 下枠を右端から短く（spin）
        trim_y=88   # 右枠を下端から短く（spin）
    ),
}

# 動画入出力
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"
output_path = "combined_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# タイムスタンプ（ISOPタイムライン）
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列がありません。")
timestamps = df['Timestamp_T=0'].values

# 50Hz系（参照コードどおり）
fps_isop = 50.948

# ---------------- ユーティリティ（レイアウトそのまま） ----------------
base_width = 600  # インプット済みコードに合わせて固定
gap_h, gap_v = 15, 40
side_margin, top_margin, bottom_margin = 20, 60, 60

def resize_to_width_and_pad(frame_bgr, target_width=base_width):
    h, w = frame_bgr.shape[:2]
    scale = target_width / w
    frame = cv2.resize(frame_bgr, (target_width, int(round(h * scale))))
    return cv2.copyMakeBorder(frame, pad, pad, pad, pad, cv2.BORDER_CONSTANT)

def _scaled_params(padded_frame, kind):
    info = INFO[kind]
    H, W = padded_frame.shape[:2]
    res_w = W - 2*pad
    res_h = H - 2*pad
    s = res_w / info["orig_w"]  # 横倍率（縦も同率）

    xy_w_res = int(round(info["xy_w"] * s))
    xy_h_res = int(round(info["xy_h"] * s))
    gap_res  = max(1, int(round(info["gap"]  * s)))

    x1 = pad + int(round(info["xy_w"] * s))
    x2 = x1 + gap_res - 1
    y1 = pad + int(round(info["xy_h"] * s))
    y2 = y1 + gap_res - 1

    # クリップ
    x1 = max(pad, min(x1, pad + res_w - 1))
    x2 = max(pad, min(x2, pad + res_w - 1))
    y1 = max(pad, min(y1, pad + res_h - 1))
    y2 = max(pad, min(y2, pad + res_h - 1))

    return dict(
        s=s, res_w=res_w, res_h=res_h,
        xy_w_res=xy_w_res, xy_h_res=xy_h_res,
        xgap=(min(x1, x2), max(x1, x2)),
        ygap=(min(y1, y2), max(y1, y2))
    )

def draw_cross_lines(padded_frame, kind):
    p = _scaled_params(padded_frame, kind)
    # 縦帯（XY-zy間）
    cv2.rectangle(padded_frame, (p["xgap"][0], pad), (p["xgap"][1], pad + p["res_h"] - 1), line_color, -1)
    # 横帯（XY-xz間）
    cv2.rectangle(padded_frame, (pad, p["ygap"][0]), (pad + p["res_w"] - 1, p["ygap"][1]), line_color, -1)

def draw_border(padded_frame, kind):
    p = _scaled_params(padded_frame, kind)
    H, W = padded_frame.shape[:2]

    # 上（パディング側）
    y_top0 = max(0, pad - border_thickness)
    y_top1 = pad - 1
    if y_top1 >= y_top0:
        cv2.rectangle(padded_frame, (pad, y_top0), (pad + p["res_w"] - 1, y_top1), line_color, -1)

    # 左（パディング側）
    x_left0 = max(0, pad - border_thickness)
    x_left1 = pad - 1
    if x_left1 >= x_left0:
        cv2.rectangle(padded_frame, (x_left0, pad), (x_left1, pad + p["res_h"] - 1), line_color, -1)

    # 右（下端から短く）
    trim_y = min(max(int(INFO[kind].get("trim_y", 12)), 0), p["res_h"] - 1)
    x_right0 = pad + p["res_w"]
    x_right1 = min(W - 1, x_right0 + border_thickness - 1)
    y_right_end = max(pad, pad + p["res_h"] - 1 - trim_y)
    if x_right1 >= x_right0 and y_right_end >= pad:
        cv2.rectangle(padded_frame, (x_right0, pad), (x_right1, y_right_end), line_color, -1)

    # 下（右端から短く）
    trim_x = min(max(int(INFO[kind].get("trim_x", 12)), 0), p["res_w"] - 1)
    y_bot0 = pad + p["res_h"]
    y_bot1 = min(H - 1, y_bot0 + border_thickness - 1)
    x_bottom_end = max(pad, pad + p["res_w"] - 1 - trim_x)
    if y_bot1 >= y_bot0 and x_bottom_end >= pad:
        cv2.rectangle(padded_frame, (pad, y_bot0), (x_bottom_end, y_bot1), line_color, -1)

def annotate_tile(frame_bgr, kind):
    """1枚のフレームをレイアウトに従って加工して返す"""
    tile = resize_to_width_and_pad(frame_bgr, base_width)
    draw_cross_lines(tile, kind)
    draw_border(tile, kind)
    return tile

def hstack_with_gap(img1, img2, gap=gap_h):
    h = max(img1.shape[0], img2.shape[0])
    g = np.zeros((h, gap, 3), dtype=np.uint8)
    img1 = cv2.copyMakeBorder(img1, 0, h - img1.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    img2 = cv2.copyMakeBorder(img2, 0, h - img2.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    return np.hstack([img1, g, img2])

def draw_labels_on_combined(combined_img, top_h):
    # （インプット済みコードの位置を踏襲）
    ci = put_text(combined_img, "xy", (side_margin + pad + 10,  top_margin + pad + 265), font_I)
    ci = put_text(ci,            "zy", (side_margin + pad + 560, top_margin + pad + 265), font_I)
    ci = put_text(ci,            "xz", (side_margin + pad + 10,  top_margin + pad + 300), font_I)
    offset_y = top_h + gap_v
    ci = put_text(ci, "xy", (side_margin + pad + 10,  top_margin + offset_y + pad + 477), font_I)
    ci = put_text(ci, "zy", (side_margin + pad + 560, top_margin + offset_y + pad + 477), font_I)
    ci = put_text(ci, "xz", (side_margin + pad + 10,  top_margin + offset_y + pad + 510), font_I)
    return ci

# === スケールバー（小数対応：端1列をアルファ合成）===
def draw_scale_bar_precise(img, top_pos, native_len, scale, height=3, color=(255,255,255)):
    """
    native_len * scale の小数を反映。
    整数部分は通常描画、端の1列は frac をアルファとして合成。
    """
    L = float(native_len) * float(scale)
    Li = int(np.floor(L))         # 整数部
    frac = L - Li                 # 0 <= frac < 1
    x, y = map(int, top_pos)

    # クリップ（高さ）
    y0 = max(0, y)
    y1 = min(img.shape[0], y + height)
    if y1 <= y0:
        return

    # 整数部
    x0 = max(0, x)
    x1 = min(img.shape[1], x + Li)
    if x1 > x0:
        cv2.rectangle(img, (x0, y0), (x1 - 1, y1 - 1), color, -1)

    # 小数部（端1列）
    x_frac = x + Li
    if frac > 1e-6 and 0 <= x_frac < img.shape[1]:
        roi = img[y0:y1, x_frac:x_frac+1].astype(np.float32)
        overlay = np.full_like(roi, np.array(color, dtype=np.float32))
        blended = roi * (1.0 - frac) + overlay * frac
        img[y0:y1, x_frac:x_frac+1] = np.clip(blended, 0, 255).astype(np.uint8)

# ---------------- キャプチャを開く ----------------
caps = {k: cv2.VideoCapture(v) for k, v in paths.items()}
for key, cap in caps.items():
    if not cap.isOpened():
        raise RuntimeError(f"❌ 開けません: {paths[key]}")
    else:
        print(f"✅ 開けました: {paths[key]}")

# フレーム数（ISOP側の共通長）と初期フレーム
count_isop_fluo  = int(caps["isop_fluo"].get(cv2.CAP_PROP_FRAME_COUNT))
count_isop_track = int(caps["isop_track"].get(cv2.CAP_PROP_FRAME_COUNT))
n_frames_isop = min(count_isop_fluo, count_isop_track, len(timestamps))
print(f"ISOP frames: fluo={count_isop_fluo}, track={count_isop_track}, timestamps={len(timestamps)} -> use {n_frames_isop}")

# SPIN同期パラメータ（参照コードどおり）
spin_start_offset   = 0.31
spin_fps            = 3.175
spin_track_max      = int(caps["spin_track"].get(cv2.CAP_PROP_FRAME_COUNT)) or 146
spin_track_end_time = (spin_track_max - 1) / spin_fps + spin_start_offset

# 直近フレームのキャッシュ（読み損ね時の保持）
last_isop_fluo  = None
last_isop_track = None
last_spin_fluo  = None
last_spin_track = None
last_spin_fluo_index  = -1
last_spin_track_index = -1

out = None

# ---------------- メインループ ----------------
for i in range(n_frames_isop):
    t = i / fps_isop
    if i % 200 == 0:
        print(f"Processing {i}/{n_frames_isop} (t={t:.3f}s)")

    # ---- ISOP fluo ----
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, f = caps["isop_fluo"].read()
    if ret: last_isop_fluo = annotate_tile(f, "isop")
    isop_fluo_tile = last_isop_fluo.copy()

    # ---- ISOP track ----
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, f = caps["isop_track"].read()
    if ret: last_isop_track = annotate_tile(f, "isop")
    isop_track_tile = last_isop_track.copy()

    # ---- SPIN fluo ----
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)

    if spin_fluo_index != last_spin_fluo_index:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, f = caps["spin_fluo"].read()
        if ret:
            last_spin_fluo = annotate_tile(f, "spin")
            last_spin_fluo_index = spin_fluo_index
    spin_fluo_tile = last_spin_fluo.copy()

    # ---- SPIN track ----
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, f = caps["spin_track"].read()
            if ret:
                last_spin_track = annotate_tile(f, "spin")
                last_spin_track_index = spin_track_index
    spin_track_tile = last_spin_track.copy()

    # ---- 見出し・タイムスタンプ（タイルに） ----
    isop_fluo_tile  = put_text(isop_fluo_tile, "ISOP microscopy", (pad + 5, pad + 5), font)
    isop_fluo_tile  = put_text(isop_fluo_tile, f"T = {timestamps[i]:.3f} s", (pad + 5, pad + 35), font)
    spin_fluo_tile  = put_text(spin_fluo_tile, "Spinning disk confocal microscopy", (pad + 5, pad + 5), font)

    # ---- 2x2結合（レイアウトはインプット済み通り）----
    top    = hstack_with_gap(isop_fluo_tile,  isop_track_tile)
    bottom = hstack_with_gap(spin_fluo_tile,  spin_track_tile)
    core   = np.vstack([top, np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8), bottom])
    combined = cv2.copyMakeBorder(core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT)

    # ---- ラベル（インプット済みの固定位置）----
    combined = draw_labels_on_combined(combined, top_h=top.shape[0])

    # ---- スケールバー（ネイティブ長→リサイズ倍率で小数まで厳密化）----
    s_isop = (isop_fluo_tile.shape[1] - 2*pad) / INFO["isop"]["orig_w"]
    s_spin = (spin_fluo_tile.shape[1] - 2*pad) / INFO["spin"]["orig_w"]

    draw_scale_bar_precise(
        combined,
        (side_margin + gap_h + pad + 420, top_margin + pad + 280),
        native_len=INFO["isop"]["scalebar_native"],
        scale=s_isop,
        height=3
    )
    draw_scale_bar_precise(
        combined,
        (side_margin + gap_h + pad + 437, top_margin + top.shape[0] + gap_v + pad + 490),
        native_len=INFO["spin"]["scalebar_native"],
        scale=s_spin,
        height=3
    )

    # ---- VideoWriter 初期化＆書き込み ----
    if out is None:
        H, W = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (W, H))
    out.write(combined)

# ---------------- 後処理 ----------------
for cap in caps.values():
    cap.release()
if out is not None:
    out.release()
print(f"✅ 出力完了: {output_path}")


✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
ISOP frames: fluo=9995, track=9995, timestamps=9996 -> use 9995
Processing 0/9995 (t=0.000s)
Processing 200/9995 (t=3.926s)
Processing 400/9995 (t=7.851s)
Processing 600/9995 (t=11.777s)
Processing 800/9995 (t=15.702s)
Processing 1000/9995 (t=19.628s)
Processing 1200/9995 (t=23.553s)
Processing 1400/9995 (t=27.479s)
Processing 1600/9995 (t=31.405s)
Processing 1800/9995 (t=35.330s)
Processing 2000/9995 (t=39.256s)
Processing 2200/9995 (t=43.181s)
Processing 2400/9995 (t=47.107s)
Processing 2600/9995 (t=51.032s)
Processing 2800/9995 (t=54.958s)
Processing 3000/9995 (t=58.884s)
Processing 3200/9995 (t=62.809s)
Processing 3400/9995 (t=6

In [104]:
#50Hz動画最新版__OK?
# === 全フレームを動画化（レイアウトはインプット済みを厳守） ===
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# ---------------- フォント ----------------
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font  = ImageFont.truetype(font_path, 25)
font_I = ImageFont.truetype(font_path_I, 30)

def put_text(img, text, position, font, color=(255,255,255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# ---------------- 共通レイアウト定数（インプット済みを厳守） ----------------
pad = 40
border_thickness = 2
line_color = (255, 255, 255)

INFO = {
    "isop": dict(
        orig_w=601, orig_h=401, xy_w=500, xy_h=300, gap=3,
        scalebar_native=48.84,
        trim_x=98,  # 下枠を右端から短く（isop）
        trim_y=98   # 右枠を下端から短く（isop）
    ),
    "spin": dict(
        orig_w=603, orig_h=603, xy_w=512, xy_h=512, gap=3,
        scalebar_native=46.51,
        trim_x=88,  # 下枠を右端から短く（spin）
        trim_y=88   # 右枠を下端から短く（spin）
    ),
}

# 動画入出力
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"
output_path = "combined_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# タイムスタンプ（ISOPタイムライン）
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列がありません。")
timestamps = df['Timestamp_T=0'].values

# 50Hz系（参照コードどおり）
fps_isop = 50.948

# ---------------- ユーティリティ（レイアウトそのまま） ----------------
base_width = 600  # インプット済みコードに合わせて固定
gap_h, gap_v = 15, 40
side_margin, top_margin, bottom_margin = 20, 60, 60

def resize_to_width_and_pad(frame_bgr, target_width=base_width):
    h, w = frame_bgr.shape[:2]
    scale = target_width / w
    frame = cv2.resize(frame_bgr, (target_width, int(round(h * scale))))
    return cv2.copyMakeBorder(frame, pad, pad, pad, pad, cv2.BORDER_CONSTANT)

def _scaled_params(padded_frame, kind):
    info = INFO[kind]
    H, W = padded_frame.shape[:2]
    res_w = W - 2*pad
    res_h = H - 2*pad
    s = res_w / info["orig_w"]  # 横倍率（縦も同率）

    xy_w_res = int(round(info["xy_w"] * s))
    xy_h_res = int(round(info["xy_h"] * s))
    gap_res  = max(1, int(round(info["gap"]  * s)))

    x1 = pad + int(round(info["xy_w"] * s))
    x2 = x1 + gap_res - 1
    y1 = pad + int(round(info["xy_h"] * s))
    y2 = y1 + gap_res - 1

    # クリップ
    x1 = max(pad, min(x1, pad + res_w - 1))
    x2 = max(pad, min(x2, pad + res_w - 1))
    y1 = max(pad, min(y1, pad + res_h - 1))
    y2 = max(pad, min(y2, pad + res_h - 1))

    return dict(
        s=s, res_w=res_w, res_h=res_h,
        xy_w_res=xy_w_res, xy_h_res=xy_h_res,
        xgap=(min(x1, x2), max(x1, x2)),
        ygap=(min(y1, y2), max(y1, y2))
    )

def draw_cross_lines(padded_frame, kind):
    p = _scaled_params(padded_frame, kind)
    # 縦帯（XY-zy間）
    cv2.rectangle(padded_frame, (p["xgap"][0], pad), (p["xgap"][1], pad + p["res_h"] - 1), line_color, -1)
    # 横帯（XY-xz間）
    cv2.rectangle(padded_frame, (pad, p["ygap"][0]), (pad + p["res_w"] - 1, p["ygap"][1]), line_color, -1)

def draw_border(padded_frame, kind):
    p = _scaled_params(padded_frame, kind)
    H, W = padded_frame.shape[:2]

    # 上（パディング側）
    y_top0 = max(0, pad - border_thickness)
    y_top1 = pad - 1
    if y_top1 >= y_top0:
        cv2.rectangle(padded_frame, (pad, y_top0), (pad + p["res_w"] - 1, y_top1), line_color, -1)

    # 左（パディング側）
    x_left0 = max(0, pad - border_thickness)
    x_left1 = pad - 1
    if x_left1 >= x_left0:
        cv2.rectangle(padded_frame, (x_left0, pad), (x_left1, pad + p["res_h"] - 1), line_color, -1)

    # 右（下端から短く）
    trim_y = min(max(int(INFO[kind].get("trim_y", 12)), 0), p["res_h"] - 1)
    x_right0 = pad + p["res_w"]
    x_right1 = min(W - 1, x_right0 + border_thickness - 1)
    y_right_end = max(pad, pad + p["res_h"] - 1 - trim_y)
    if x_right1 >= x_right0 and y_right_end >= pad:
        cv2.rectangle(padded_frame, (x_right0, pad), (x_right1, y_right_end), line_color, -1)

    # 下（右端から短く）
    trim_x = min(max(int(INFO[kind].get("trim_x", 12)), 0), p["res_w"] - 1)
    y_bot0 = pad + p["res_h"]
    y_bot1 = min(H - 1, y_bot0 + border_thickness - 1)
    x_bottom_end = max(pad, pad + p["res_w"] - 1 - trim_x)
    if y_bot1 >= y_bot0 and x_bottom_end >= pad:
        cv2.rectangle(padded_frame, (pad, y_bot0), (x_bottom_end, y_bot1), line_color, -1)

def annotate_tile(frame_bgr, kind):
    """1枚のフレームをレイアウトに従って加工して返す"""
    tile = resize_to_width_and_pad(frame_bgr, base_width)
    draw_cross_lines(tile, kind)
    draw_border(tile, kind)
    return tile

def hstack_with_gap(img1, img2, gap=gap_h):
    h = max(img1.shape[0], img2.shape[0])
    g = np.zeros((h, gap, 3), dtype=np.uint8)
    img1 = cv2.copyMakeBorder(img1, 0, h - img1.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    img2 = cv2.copyMakeBorder(img2, 0, h - img2.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    return np.hstack([img1, g, img2])

def draw_labels_on_combined(combined_img, top_h):
    # （インプット済みコードの位置を踏襲）
    ci = put_text(combined_img, "xy", (side_margin + pad + 10,  top_margin + pad + 265), font_I)
    ci = put_text(ci,            "zy", (side_margin + pad + 560, top_margin + pad + 265), font_I)
    ci = put_text(ci,            "xz", (side_margin + pad + 10,  top_margin + pad + 300), font_I)
    offset_y = top_h + gap_v
    ci = put_text(ci, "xy", (side_margin + pad + 10,  top_margin + offset_y + pad + 477), font_I)
    ci = put_text(ci, "zy", (side_margin + pad + 560, top_margin + offset_y + pad + 477), font_I)
    ci = put_text(ci, "xz", (side_margin + pad + 10,  top_margin + offset_y + pad + 510), font_I)
    return ci

def draw_scale_bar(img, top_pos, length_px):
    x, y = top_pos
    cv2.rectangle(img, (x, y), (x + int(round(length_px)), y + 3), (255, 255, 255), -1)

# ---------------- キャプチャを開く ----------------
caps = {k: cv2.VideoCapture(v) for k, v in paths.items()}
for key, cap in caps.items():
    if not cap.isOpened():
        raise RuntimeError(f"❌ 開けません: {paths[key]}")
    else:
        print(f"✅ 開けました: {paths[key]}")

# フレーム数（ISOP側の共通長）と初期フレーム
count_isop_fluo  = int(caps["isop_fluo"].get(cv2.CAP_PROP_FRAME_COUNT))
count_isop_track = int(caps["isop_track"].get(cv2.CAP_PROP_FRAME_COUNT))
n_frames_isop = min(count_isop_fluo, count_isop_track, len(timestamps))
print(f"ISOP frames: fluo={count_isop_fluo}, track={count_isop_track}, timestamps={len(timestamps)} -> use {n_frames_isop}")

# SPIN同期パラメータ（参照コードどおり）
spin_start_offset   = 0.31
spin_fps            = 3.175
spin_track_max      = int(caps["spin_track"].get(cv2.CAP_PROP_FRAME_COUNT)) or 146
spin_track_end_time = (spin_track_max - 1) / spin_fps + spin_start_offset

# 直近フレームのキャッシュ（読み損ね時の保持）
last_isop_fluo  = None
last_isop_track = None
last_spin_fluo  = None
last_spin_track = None
last_spin_fluo_index  = -1
last_spin_track_index = -1

out = None

# ---------------- メインループ ----------------
for i in range(n_frames_isop):
    t = i / fps_isop
    if i % 200 == 0:
        print(f"Processing {i}/{n_frames_isop} (t={t:.3f}s)")

    # ---- ISOP fluo ----
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, f = caps["isop_fluo"].read()
    if ret: last_isop_fluo = annotate_tile(f, "isop")
    isop_fluo_tile = last_isop_fluo.copy()

    # ---- ISOP track ----
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, f = caps["isop_track"].read()
    if ret: last_isop_track = annotate_tile(f, "isop")
    isop_track_tile = last_isop_track.copy()

    # ---- SPIN fluo ----
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)

    if spin_fluo_index != last_spin_fluo_index:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, f = caps["spin_fluo"].read()
        if ret:
            last_spin_fluo = annotate_tile(f, "spin")
            last_spin_fluo_index = spin_fluo_index
    spin_fluo_tile = last_spin_fluo.copy()

    # ---- SPIN track ----
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, f = caps["spin_track"].read()
            if ret:
                last_spin_track = annotate_tile(f, "spin")
                last_spin_track_index = spin_track_index
    spin_track_tile = last_spin_track.copy()

    # ---- 見出し・タイムスタンプ（タイルに） ----
    isop_fluo_tile  = put_text(isop_fluo_tile, "ISOP microscopy", (pad + 5, pad + 5), font)
    isop_fluo_tile  = put_text(isop_fluo_tile, f"T = {timestamps[i]:.3f} s", (pad + 5, pad + 35), font)
    spin_fluo_tile  = put_text(spin_fluo_tile, "Spinning disk confocal microscopy", (pad + 5, pad + 5), font)

    # ---- 2x2結合（レイアウトはインプット済み通り）----
    top    = hstack_with_gap(isop_fluo_tile,  isop_track_tile)
    bottom = hstack_with_gap(spin_fluo_tile,  spin_track_tile)
    core   = np.vstack([top, np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8), bottom])
    combined = cv2.copyMakeBorder(core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT)

    # ---- ラベル（インプット済みの固定位置）----
    combined = draw_labels_on_combined(combined, top_h=top.shape[0])

    # ---- スケールバー（ネイティブ長→リサイズ倍率で厳密化）----
    s_isop = (isop_fluo_tile.shape[1] - 2*pad) / INFO["isop"]["orig_w"]
    s_spin = (spin_fluo_tile.shape[1] - 2*pad) / INFO["spin"]["orig_w"]
    len_isop = max(1, int(round(INFO["isop"]["scalebar_native"] * s_isop)))
    len_spin = max(1, int(round(INFO["spin"]["scalebar_native"] * s_spin)))

    draw_scale_bar(combined, (side_margin + gap_h + pad + 420,                        top_margin + pad + 280),                             len_isop)  # 上段（ISOP）
    draw_scale_bar(combined, (side_margin + gap_h + pad + 437, top_margin + top.shape[0] + gap_v + pad + 490), len_spin)  # 下段（SPIN）

    # ---- VideoWriter 初期化＆書き込み ----
    if out is None:
        H, W = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (W, H))
    out.write(combined)

# ---------------- 後処理 ----------------
for cap in caps.values():
    cap.release()
if out is not None:
    out.release()
print(f"✅ 出力完了: {output_path}")


✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
ISOP frames: fluo=9995, track=9995, timestamps=9996 -> use 9995
Processing 0/9995 (t=0.000s)
Processing 200/9995 (t=3.926s)
Processing 400/9995 (t=7.851s)
Processing 600/9995 (t=11.777s)
Processing 800/9995 (t=15.702s)
Processing 1000/9995 (t=19.628s)
Processing 1200/9995 (t=23.553s)
Processing 1400/9995 (t=27.479s)
Processing 1600/9995 (t=31.405s)
Processing 1800/9995 (t=35.330s)
Processing 2000/9995 (t=39.256s)
Processing 2200/9995 (t=43.181s)
Processing 2400/9995 (t=47.107s)
Processing 2600/9995 (t=51.032s)
Processing 2800/9995 (t=54.958s)
Processing 3000/9995 (t=58.884s)
Processing 3200/9995 (t=62.809s)
Processing 3400/9995 (t=6

In [105]:
#画像出力、線は食い込まないように調整済み, スケールバー小数点以下を反映
import cv2
import numpy as np
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font = ImageFont.truetype(font_path, 25)
font_I = ImageFont.truetype(font_path_I, 30)

def put_text(img, text, position, font, color=(255,255,255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 共通パラメータ ===
pad = 40
border_thickness = 2
line_color = (255, 255, 255)

# === パス設定 ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"
df = pd.read_csv(timestamp_csv_path)
timestamps = df["Timestamp_T=0"].values

# === フレーム取得 & パディング ===
def read_frame_and_pad(path, target_width):
    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return np.zeros((600 + 2*pad, target_width + 2*pad, 3), dtype=np.uint8)
    h, w = frame.shape[:2]
    scale = target_width / w
    frame = cv2.resize(frame, (target_width, int(round(h * scale))))
    return cv2.copyMakeBorder(frame, pad, pad, pad, pad, cv2.BORDER_CONSTANT)

base_width = 600
frames = {k: read_frame_and_pad(v, base_width) for k, v in paths.items()}

# === レイアウト定義（ネイティブ画素）===
def _scaled_params(frame, kind):
    """パディング後フレームから、スケールとXY/ギャップ位置（リサイズ後）を算出"""
    info = INFO[kind]
    H, W = frame.shape[:2]
    res_w = W - 2*pad
    res_h = H - 2*pad
    s = res_w / info["orig_w"]  # 横方向スケール（縦も同倍率）

    xy_w_res = int(round(info["xy_w"] * s))
    xy_h_res = int(round(info["xy_h"] * s))
    gap_res  = max(1, int(round(info["gap"]  * s)))

    # ギャップ帯（リサイズ後・パディング込み座標）
    x1 = pad + int(round(info["xy_w"] * s))
    x2 = x1 + gap_res - 1
    y1 = pad + int(round(info["xy_h"] * s))
    y2 = y1 + gap_res - 1

    # クリップ
    x1 = max(pad, min(x1, pad + res_w - 1))
    x2 = max(pad, min(x2, pad + res_w - 1))
    y1 = max(pad, min(y1, pad + res_h - 1))
    y2 = max(pad, min(y2, pad + res_h - 1))

    return dict(
        s=s, res_w=res_w, res_h=res_h,
        xy_w_res=xy_w_res, xy_h_res=xy_h_res,
        xgap=(min(x1, x2), max(x1, x2)),
        ygap=(min(y1, y2), max(y1, y2))
    )

# === クロスライン：ギャップ帯の中だけに描く ===
def draw_cross_lines(frame, kind):
    p = _scaled_params(frame, kind)
    cv2.rectangle(frame, (p["xgap"][0], pad), (p["xgap"][1], pad + p["res_h"] - 1), line_color, -1)
    cv2.rectangle(frame, (pad, p["ygap"][0]), (pad + p["res_w"] - 1, p["ygap"][1]), line_color, -1)

# === 角欠き値込みのINFO（isop/spin個別設定）===
INFO = {
    "isop": dict(
        orig_w=601, orig_h=401, xy_w=500, xy_h=300, gap=3,
        scalebar_native=48.84,
        trim_x=98,   # 下枠を右端から短く
        trim_y=98    # 右枠を下端から短く
    ),
    "spin": dict(
        orig_w=603, orig_h=603, xy_w=512, xy_h=512, gap=3,
        scalebar_native=46.51,
        trim_x=88,
        trim_y=88
    ),
}

def draw_border(frame, kind):
    p = _scaled_params(frame, kind)
    H, W = frame.shape[:2]

    # 上（パディング側）
    y_top0 = max(0, pad - border_thickness)
    y_top1 = pad - 1
    if y_top1 >= y_top0:
        cv2.rectangle(frame, (pad, y_top0), (pad + p["res_w"] - 1, y_top1), line_color, -1)

    # 左（パディング側）
    x_left0 = max(0, pad - border_thickness)
    x_left1 = pad - 1
    if x_left1 >= x_left0:
        cv2.rectangle(frame, (x_left0, pad), (x_left1, pad + p["res_h"] - 1), line_color, -1)

    # 右（下端から短く）
    trim_y = min(max(int(INFO[kind].get("trim_y", 12)), 0), p["res_h"] - 1)
    x_right0 = pad + p["res_w"]
    x_right1 = min(W - 1, x_right0 + border_thickness - 1)
    y_right_end = max(pad, pad + p["res_h"] - 1 - trim_y)
    if x_right1 >= x_right0 and y_right_end >= pad:
        cv2.rectangle(frame, (x_right0, pad), (x_right1, y_right_end), line_color, -1)

    # 下（右端から短く）
    trim_x = min(max(int(INFO[kind].get("trim_x", 12)), 0), p["res_w"] - 1)
    y_bot0 = pad + p["res_h"]
    y_bot1 = min(H - 1, y_bot0 + border_thickness - 1)
    x_bottom_end = max(pad, pad + p["res_w"] - 1 - trim_x)
    if y_bot1 >= y_bot0 and x_bottom_end >= pad:
        cv2.rectangle(frame, (pad, y_bot0), (x_bottom_end, y_bot1), line_color, -1)

# === 描画実行（静止画）===
for key in frames:
    kind = "isop" if "isop" in key else "spin"
    draw_cross_lines(frames[key], kind)
    draw_border(frames[key], kind)

# === ラベル・スケールバー（元の配置は維持）===
frames["isop_fluo"] = put_text(frames["isop_fluo"], "ISOP microscopy", (pad + 5, pad + 5), font)
frames["isop_fluo"] = put_text(frames["isop_fluo"], f"T = {timestamps[0]:.3f} s", (pad + 5, pad + 35), font)
frames["spin_fluo"] = put_text(frames["spin_fluo"], "Spinning disk confocal microscopy", (pad + 5, pad + 5), font)

# === 上下結合 ===
gap_h, gap_v = 15, 40
side_margin, top_margin, bottom_margin = 20, 60, 60

def hstack_with_gap(img1, img2, gap=gap_h):
    h = max(img1.shape[0], img2.shape[0])
    g = np.zeros((h, gap, 3), dtype=np.uint8)
    img1 = cv2.copyMakeBorder(img1, 0, h - img1.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    img2 = cv2.copyMakeBorder(img2, 0, h - img2.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    return np.hstack([img1, g, img2])

top = hstack_with_gap(frames["isop_fluo"], frames["isop_track"])
bottom = hstack_with_gap(frames["spin_fluo"], frames["spin_track"])
core = np.vstack([top, np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8), bottom])
combined = cv2.copyMakeBorder(core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT)

# === ラベル（元の位置のまま）===
def draw_labels_on_combined(combined_img):
    combined_img = put_text(combined_img, "xy", (side_margin + pad + 10, top_margin + pad + 265), font_I)
    combined_img = put_text(combined_img, "zy", (side_margin + pad + 560, top_margin + pad + 265), font_I)
    combined_img = put_text(combined_img, "xz", (side_margin + pad + 10, top_margin + pad + 300), font_I)
    offset_y = top.shape[0] + gap_v
    combined_img = put_text(combined_img, "xy", (side_margin + pad + 10, top_margin + offset_y + pad + 477), font_I)
    combined_img = put_text(combined_img, "zy", (side_margin + pad + 560, top_margin + offset_y + pad + 477), font_I)
    combined_img = put_text(combined_img, "xz", (side_margin + pad + 10, top_margin + offset_y + pad + 510), font_I)
    return combined_img

combined = draw_labels_on_combined(combined)

# === スケールバー（小数対応：端1列をアルファ合成）===
def draw_scale_bar_precise(img, top_pos, native_len, scale, height=3, color=(255,255,255)):
    """
    native_len * scale の小数を反映。
    整数部分は通常描画、端の1列は frac をアルファとして合成。
    """
    L = float(native_len) * float(scale)
    Li = int(np.floor(L))         # 整数部
    frac = L - Li                 # 0 <= frac < 1
    x, y = map(int, top_pos)

    # クリップ（高さ）
    y0 = max(0, y)
    y1 = min(img.shape[0], y + height)
    if y1 <= y0:
        return

    # 整数部
    x0 = max(0, x)
    x1 = min(img.shape[1], x + Li)
    if x1 > x0:
        cv2.rectangle(img, (x0, y0), (x1 - 1, y1 - 1), color, -1)

    # 小数部（端1列）
    x_frac = x + Li
    if frac > 1e-6 and 0 <= x_frac < img.shape[1]:
        roi = img[y0:y1, x_frac:x_frac+1].astype(np.float32)
        overlay = np.full_like(roi, np.array(color, dtype=np.float32))
        blended = roi * (1.0 - frac) + overlay * frac
        img[y0:y1, x_frac:x_frac+1] = np.clip(blended, 0, 255).astype(np.uint8)

# リサイズ倍率（タイル見かけ幅から算出）
s_isop = (frames["isop_fluo"].shape[1] - 2*pad) / INFO["isop"]["orig_w"]  # = base_width / 601
s_spin = (frames["spin_fluo"].shape[1] - 2*pad) / INFO["spin"]["orig_w"]  # = base_width / 603

# 位置は従来どおり
draw_scale_bar_precise(combined, (side_margin + gap_h + pad + 420, top_margin + pad + 280),
                       native_len=INFO["isop"]["scalebar_native"], scale=s_isop, height=3)
draw_scale_bar_precise(combined, (side_margin + gap_h + pad + 437, top_margin + top.shape[0] + gap_v + pad + 490),
                       native_len=INFO["spin"]["scalebar_native"], scale=s_spin, height=3)

# === 保存 ===
cv2.imwrite("combined_frame_cleaned_fixed.png", combined)
print("✅ 完了: combined_frame_cleaned_fixed.png")


✅ 完了: combined_frame_cleaned_fixed.png


In [103]:
#画像出力OK、線は食い込まないように調整済み
import cv2
import numpy as np
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font = ImageFont.truetype(font_path, 25)
font_I = ImageFont.truetype(font_path_I, 30)

def put_text(img, text, position, font, color=(255,255,255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 共通パラメータ ===
pad = 40
border_thickness = 2
line_color = (255, 255, 255)

# === レイアウト定義（ネイティブ画素）===
#INFO = {
 #   "isop": dict(orig_w=601, orig_h=401, xy_w=500, xy_h=300, gap=3, scalebar_native=48.84),
  #  "spin": dict(orig_w=603, orig_h=603, xy_w=512, xy_h=512, gap=3, scalebar_native=46.51),
#}

# === パス設定 ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"
df = pd.read_csv(timestamp_csv_path)
timestamps = df["Timestamp_T=0"].values

# === フレーム取得 & パディング ===
def read_frame_and_pad(path, target_width):
    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return np.zeros((600 + 2*pad, target_width + 2*pad, 3), dtype=np.uint8)
    h, w = frame.shape[:2]
    scale = target_width / w
    frame = cv2.resize(frame, (target_width, int(round(h * scale))))
    return cv2.copyMakeBorder(frame, pad, pad, pad, pad, cv2.BORDER_CONSTANT)

base_width = 600
frames = {k: read_frame_and_pad(v, base_width) for k, v in paths.items()}

def _scaled_params(frame, kind):
    """パディング後フレームから、スケールとXY/ギャップ位置（リサイズ後）を算出"""
    info = INFO[kind]
    H, W = frame.shape[:2]
    # パディング除去後の中身サイズ
    res_w = W - 2*pad
    res_h = H - 2*pad
    s = res_w / info["orig_w"]  # 横方向スケール（縦も同倍率）

    xy_w_res = int(round(info["xy_w"] * s))
    xy_h_res = int(round(info["xy_h"] * s))
    gap_res  = max(1, int(round(info["gap"]  * s)))

    # ギャップ帯（リサイズ後・パディング込み座標）
    x1 = pad + int(round(info["xy_w"] * s))
    x2 = x1 + gap_res - 1
    y1 = pad + int(round(info["xy_h"] * s))
    y2 = y1 + gap_res - 1

    # クリップ
    x1 = max(pad, min(x1, pad + res_w - 1))
    x2 = max(pad, min(x2, pad + res_w - 1))
    y1 = max(pad, min(y1, pad + res_h - 1))
    y2 = max(pad, min(y2, pad + res_h - 1))

    return dict(
        s=s, res_w=res_w, res_h=res_h,
        xy_w_res=xy_w_res, xy_h_res=xy_h_res,
        xgap=(min(x1, x2), max(x1, x2)),
        ygap=(min(y1, y2), max(y1, y2))
    )

# === クロスライン：ギャップ帯の中だけに描く ===
def draw_cross_lines(frame, kind):
    p = _scaled_params(frame, kind)
    # 縦帯（XY-zy間）：全高に延ばすが、帯はギャップ内のみ
    cv2.rectangle(frame, (p["xgap"][0], pad), (p["xgap"][1], pad + p["res_h"] - 1), line_color, -1)
    # 横帯（XY-xz間）：全幅に延ばすが、帯はギャップ内のみ
    cv2.rectangle(frame, (pad, p["ygap"][0]), (pad + p["res_w"] - 1, p["ygap"][1]), line_color, -1)

    
INFO = {
    "isop": dict(
        orig_w=601, orig_h=401, xy_w=500, xy_h=300, gap=3,
        scalebar_native=48.84,
        trim_x=98,   # 下枠を右端から何px短くするか（isop用）
        trim_y=98    # 右枠を下端から何px短くするか（isop用）
    ),
    "spin": dict(
        orig_w=603, orig_h=603, xy_w=512, xy_h=512, gap=3,
        scalebar_native=46.51,
        trim_x=88,   # 下枠を右端から（spin用）
        trim_y=88    # 右枠を下端から（spin用）
    ),
}
 
    
    
def draw_border(frame, kind):
    p = _scaled_params(frame, kind)
    H, W = frame.shape[:2]

    # 上（パディング側）
    y_top0 = max(0, pad - border_thickness)
    y_top1 = pad - 1
    if y_top1 >= y_top0:
        cv2.rectangle(frame, (pad, y_top0), (pad + p["res_w"] - 1, y_top1), line_color, -1)

    # 左（パディング側）
    x_left0 = max(0, pad - border_thickness)
    x_left1 = pad - 1
    if x_left1 >= x_left0:
        cv2.rectangle(frame, (x_left0, pad), (x_left1, pad + p["res_h"] - 1), line_color, -1)

    # --- ここから角欠き（kind別設定） ---
    trim_x_cfg = max(0, int(INFO[kind].get("trim_x", 12)))
    trim_y_cfg = max(0, int(INFO[kind].get("trim_y", 12)))

    # 右（パディング側）—下端から trim_y_cfg 分だけ短く
    x_right0 = pad + p["res_w"]
    x_right1 = min(W - 1, x_right0 + border_thickness - 1)
    trim_y = min(trim_y_cfg, p["res_h"] - 1)  # 安全クランプ
    y_right_end = pad + p["res_h"] - 1 - trim_y
    y_right_end = max(pad, y_right_end)
    if x_right1 >= x_right0 and y_right_end >= pad:
        cv2.rectangle(frame, (x_right0, pad), (x_right1, y_right_end), line_color, -1)

    # 下（パディング側）—右端から trim_x_cfg 分だけ短く
    y_bot0 = pad + p["res_h"]
    y_bot1 = min(H - 1, y_bot0 + border_thickness - 1)
    trim_x = min(trim_x_cfg, p["res_w"] - 1)  # 安全クランプ
    x_bottom_end = pad + p["res_w"] - 1 - trim_x
    x_bottom_end = max(pad, x_bottom_end)
    if y_bot1 >= y_bot0 and x_bottom_end >= pad:
        cv2.rectangle(frame, (pad, y_bot0), (x_bottom_end, y_bot1), line_color, -1)
    
    
# # 角から削る量（px）—必要に応じて調整。まずは 12 前後から試すと見やすいです。
# corner_trim_x = 98   # 下枠を「右端から」何px短くするか
# corner_trim_y = 98   # 右枠を「下端から」何px短くするか

# # === 外枠：4辺ともパディング側に描く（右下は“欠く”）===
# def draw_border(frame, kind):
#     p = _scaled_params(frame, kind)
#     H, W = frame.shape[:2]

#     # 上（パディング側）
#     y_top0 = max(0, pad - border_thickness)
#     y_top1 = pad - 1
#     if y_top1 >= y_top0:
#         cv2.rectangle(frame, (pad, y_top0), (pad + p["res_w"] - 1, y_top1), line_color, -1)

#     # 左（パディング側）
#     x_left0 = max(0, pad - border_thickness)
#     x_left1 = pad - 1
#     if x_left1 >= x_left0:
#         cv2.rectangle(frame, (x_left0, pad), (x_left1, pad + p["res_h"] - 1), line_color, -1)

#     # 右（パディング側）—下端から corner_trim_y 分だけ短くする
#     x_right0 = pad + p["res_w"]
#     x_right1 = min(W - 1, x_right0 + border_thickness - 1)
#     trim_y = min(max(corner_trim_y, 0), p["res_h"] - 1)  # 安全クランプ
#     y_right_end = pad + p["res_h"] - 1 - trim_y
#     y_right_end = max(pad, y_right_end)  # はみ出し防止
#     if x_right1 >= x_right0 and y_right_end >= pad:
#         cv2.rectangle(frame, (x_right0, pad), (x_right1, y_right_end), line_color, -1)

#     # 下（パディング側）—右端から corner_trim_x 分だけ短くする
#     y_bot0 = pad + p["res_h"]
#     y_bot1 = min(H - 1, y_bot0 + border_thickness - 1)
#     trim_x = min(max(corner_trim_x, 0), p["res_w"] - 1)  # 安全クランプ
#     x_bottom_end = pad + p["res_w"] - 1 - trim_x
#     x_bottom_end = max(pad, x_bottom_end)  # はみ出し防止
#     if y_bot1 >= y_bot0 and x_bottom_end >= pad:
#         cv2.rectangle(frame, (pad, y_bot0), (x_bottom_end, y_bot1), line_color, -1)
 
    
    
    
#     # === 外枠：4辺ともパディング側に描く ＋ 右下角だけ黒で“角落とし”する ===
# def draw_border(frame, kind):
#     p = _scaled_params(frame, kind)
#     H, W = frame.shape[:2]

#     # 上（pad のすぐ上側）
#     y_top0 = max(0, pad - border_thickness)
#     y_top1 = pad - 1
#     if y_top1 >= y_top0:
#         cv2.rectangle(frame, (pad, y_top0), (pad + p["res_w"] - 1, y_top1), line_color, -1)

#     # 左（pad のすぐ左側）
#     x_left0 = max(0, pad - border_thickness)
#     x_left1 = pad - 1
#     if x_left1 >= x_left0:
#         cv2.rectangle(frame, (x_left0, pad), (x_left1, pad + p["res_h"] - 1), line_color, -1)

#     # 右（内容右端の外＝右パディング側）
#     x_right0 = pad + p["res_w"]
#     x_right1 = min(W - 1, x_right0 + border_thickness - 1)
#     if x_right1 >= x_right0:
#         cv2.rectangle(frame, (x_right0, pad), (x_right1, pad + p["res_h"] - 1), line_color, -1)

#     # 下（内容下端の外＝下パディング側）
#     y_bot0 = pad + p["res_h"]
#     y_bot1 = min(H - 1, y_bot0 + border_thickness - 1)
#     if y_bot1 >= y_bot0:
#         cv2.rectangle(frame, (pad, y_bot0), (pad + p["res_w"] - 1, y_bot1), line_color, -1)

#     # ← ここがポイント：右下角の交差部分だけ黒で埋め戻して“角”を消す
#     if x_right1 >= x_right0 and y_bot1 >= y_bot0:
#         cv2.rectangle(frame, (x_right0, y_bot0), (x_right1, y_bot1), (0, 0, 0), -1)
    
    
    
# # === 外枠：4辺ともパディング側に描く（中身に食い込まない & クロスと重ならない）===
# def draw_border(frame, kind):
#     p = _scaled_params(frame, kind)
#     H, W = frame.shape[:2]

#     # 上（pad のすぐ上側）
#     y_top0 = max(0, pad - border_thickness)
#     y_top1 = pad - 1
#     if y_top1 >= y_top0:
#         cv2.rectangle(frame, (pad, y_top0), (pad + p["res_w"] - 1, y_top1), line_color, -1)

#     # 左（pad のすぐ左側）
#     x_left0 = max(0, pad - border_thickness)
#     x_left1 = pad - 1
#     if x_left1 >= x_left0:
#         cv2.rectangle(frame, (x_left0, pad), (x_left1, pad + p["res_h"] - 1), line_color, -1)

#     # 右（内容の右端の外＝右パディング側）
#     x_right0 = pad + p["res_w"]
#     x_right1 = min(W - 1, x_right0 + border_thickness - 1)
#     if x_right1 >= x_right0:
#         cv2.rectangle(frame, (x_right0, pad), (x_right1, pad + p["res_h"] - 1), line_color, -1)

#     # 下（内容の下端の外＝下パディング側）
#     y_bot0 = pad + p["res_h"]
#     y_bot1 = min(H - 1, y_bot0 + border_thickness - 1)
#     if y_bot1 >= y_bot0:
#         cv2.rectangle(frame, (pad, y_bot0), (pad + p["res_w"] - 1, y_bot1), line_color, -1)
    
    
    
# === 外枠：上・左はパディング側、下・右はギャップ帯の中に描く（中身に食い込まない）===
# def draw_border(frame, kind):
#     p = _scaled_params(frame, kind)
#     H, W = frame.shape[:2]

#     # 上枠（パディング側：y ∈ [pad - border_thickness, pad - 1]）
#     y_top0 = max(0, pad - border_thickness)
#     cv2.rectangle(frame, (pad, y_top0), (pad + p["res_w"] - 1, pad - 1), line_color, -1)

#     # 左枠（パディング側：x ∈ [pad - border_thickness, pad - 1]）
#     x_left0 = max(0, pad - border_thickness)
#     cv2.rectangle(frame, (x_left0, pad), (pad - 1, pad + p["res_h"] - 1), line_color, -1)

#     # 下枠（XYの下側だけ、ギャップ帯内）
#     cv2.rectangle(
#         frame,
#         (pad, p["ygap"][0]),
#         (pad + p["xy_w_res"] - 1, p["ygap"][1]),
#         line_color, -1
#     )

#     # 右枠（XYの右側だけ、ギャップ帯内）
#     cv2.rectangle(
#         frame,
#         (p["xgap"][0], pad),
#         (p["xgap"][1], pad + p["xy_h_res"] - 1),
#         line_color, -1
#     )

# === 描画実行 ===
for key in frames:
    kind = "isop" if "isop" in key else "spin"
    draw_cross_lines(frames[key], kind)
    draw_border(frames[key], kind)

# === ラベル・スケールバー（元の配置は維持、長さだけ厳密化）===
frames["isop_fluo"] = put_text(frames["isop_fluo"], "ISOP microscopy", (pad + 5, pad + 5), font)
frames["isop_fluo"] = put_text(frames["isop_fluo"], f"T = {timestamps[0]:.3f} s", (pad + 5, pad + 35), font)
frames["spin_fluo"] = put_text(frames["spin_fluo"], "Spinning disk confocal microscopy", (pad + 5, pad + 5), font)

# === 上下結合 ===
gap_h, gap_v = 15, 40
side_margin, top_margin, bottom_margin = 20, 60, 60

def hstack_with_gap(img1, img2, gap=gap_h):
    h = max(img1.shape[0], img2.shape[0])
    g = np.zeros((h, gap, 3), dtype=np.uint8)
    img1 = cv2.copyMakeBorder(img1, 0, h - img1.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    img2 = cv2.copyMakeBorder(img2, 0, h - img2.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    return np.hstack([img1, g, img2])

top = hstack_with_gap(frames["isop_fluo"], frames["isop_track"])
bottom = hstack_with_gap(frames["spin_fluo"], frames["spin_track"])
core = np.vstack([top, np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8), bottom])
combined = cv2.copyMakeBorder(core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT)

# === ラベル（元の位置のまま）===
def draw_labels_on_combined(combined_img):
    combined_img = put_text(combined_img, "xy", (side_margin + pad + 10, top_margin + pad + 265), font_I)
    combined_img = put_text(combined_img, "zy", (side_margin + pad + 560, top_margin + pad + 265), font_I)
    combined_img = put_text(combined_img, "xz", (side_margin + pad + 10, top_margin + pad + 300), font_I)
    offset_y = top.shape[0] + gap_v
    combined_img = put_text(combined_img, "xy", (side_margin + pad + 10, top_margin + offset_y + pad + 477), font_I)
    combined_img = put_text(combined_img, "zy", (side_margin + pad + 560, top_margin + offset_y + pad + 477), font_I)
    combined_img = put_text(combined_img, "xz", (side_margin + pad + 10, top_margin + offset_y + pad + 510), font_I)
    return combined_img

combined = draw_labels_on_combined(combined)

# === スケールバー ===
def draw_scale_bar(img, top_pos, length_px):
    x, y = top_pos
    cv2.rectangle(img, (x, y), (x + int(round(length_px)), y + 3), (255, 255, 255), -1)

# リサイズ倍率から厳密長を算出
s_isop = (frames["isop_fluo"].shape[1] - 2*pad) / INFO["isop"]["orig_w"]  # = base_width / 601
s_spin = (frames["spin_fluo"].shape[1] - 2*pad) / INFO["spin"]["orig_w"]  # = base_width / 603
len_isop = max(1, int(round(INFO["isop"]["scalebar_native"] * s_isop)))   # ≈ 48.84 * s_isop
len_spin = max(1, int(round(INFO["spin"]["scalebar_native"] * s_spin)))   # ≈ 46.51 * s_spin

# 位置は従来どおり（必要ならここだけ数値調整）
draw_scale_bar(combined, (side_margin + gap_h + pad + 420, top_margin + pad + 280), len_isop)   # 上段（ISOP）
draw_scale_bar(combined, (side_margin + gap_h + pad + 437, top_margin + top.shape[0] + gap_v + pad + 490), len_spin)  # 下段（SPIN）

# === 保存 ===
cv2.imwrite("combined_frame_cleaned_fixed.png", combined)
print("✅ 完了: combined_frame_cleaned_fixed.png")


✅ 完了: combined_frame_cleaned_fixed.png


In [93]:
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)

def put_text_pil(img, text, position, font_obj, color=(255, 255, 255)):
    """OpenCV画像にPILでテキスト描画"""
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_obj, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial(img, text, position, color=(255, 255, 255)):
    return put_text_pil(img, text, position, font, color)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    return put_text_pil(img, text, position, font_I, color)

# === リサイズ（横幅指定） ===
def resize_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(round(h * scale))
    return cv2.resize(image, (target_width, new_h))

# === 汎用：3面図ギャップと外枠、座標ラベル、スケールバーを安全に描く ===
def annotate_orthoview_tile(tile_bgr,
                            orig_w, orig_h,
                            xy_w, xy_h,  # XYの幅・高さ（ネイティブ）
                            gap=3,
                            pad_px=2,
                            border_thickness=2,
                            line_color=(255,255,255),
                            draw_labels=True,
                            label_color=(255,255,255),
                            # ← 追加：ラベルオフセット(px, リサイズ後・パディング前)
                            label_offsets_px=None,  # dict like {'xy':(dx,dy), 'yz':(dx,dy), 'xz':(dx,dy)}
                            # スケールバー
                            scalebar_px_native=None,
                            scalebar_height=3,
                            scalebar_offset=(10, 10)  # XY左下から
                            ):
    h, w = tile_bgr.shape[:2]
    sx = w / float(orig_w)
    sy = h / float(orig_h)

    # === ギャップのネイティブ座標
    gap_x1_native = xy_w
    gap_x2_native = xy_w + gap - 1
    gap_y1_native = xy_h
    gap_y2_native = xy_h + gap - 1

    # スケールして整数化
    gx1 = int(round(gap_x1_native * sx))
    gx2 = int(round(gap_x2_native * sx))
    gy1 = int(round(gap_y1_native * sy))
    gy2 = int(round(gap_y2_native * sy))
    gx1, gx2 = np.clip([gx1, gx2], 0, w-1)
    gy1, gy2 = np.clip([gy1, gy2], 0, h-1)
    if gx2 < gx1: gx1, gx2 = gx2, gx1
    if gy2 < gy1: gy1, gy2 = gy2, gy1

    # === ギャップにのみ白帯
    cv2.rectangle(tile_bgr, (gx1, 0), (gx2, h-1), line_color, -1)  # 縦帯（XY-zy間）
    cv2.rectangle(tile_bgr, (0, gy1), (w-1, gy2), line_color, -1)  # 横帯（XY-xz間）

    # === ラベル（基準点＋オフセット）
    if draw_labels:
        # パネル位置（リサイズ後）
        xy_w_res = int(round(xy_w * sx))
        xy_h_res = int(round(xy_h * sy))
        zy_x0 = int(round((xy_w + gap) * sx))  # ZY左上 x
        xz_y0 = int(round((xy_h + gap) * sy))  # XZ左上 y

        # デフォルトのオフセット（必要ならここで初期値変更）
        default_offsets = {'xy': (10, -35), 'yz': (10, 20), 'xz': (10, 20)}
        if label_offsets_px is None:
            label_offsets_px = default_offsets
        else:
            # 足りないキーはデフォルトで補完
            for k, v in default_offsets.items():
                label_offsets_px.setdefault(k, v)

        # 基準点
        anchors = {
            'xy': (0, xy_h_res),       # 左下
            'yz': (zy_x0, 0),          # 左上
            'xz': (0, xz_y0),          # 左上
        }

        # 反映（基準点＋オフセット）
        for key in ['xy', 'yz', 'xz']:
            ax, ay = anchors[key]
            dx, dy = label_offsets_px[key]
            pos = (max(0, ax + int(dx)), max(0, ay + int(dy)))
            tile_bgr = put_text_arial_italic(tile_bgr, key, pos, label_color)

    # === スケールバー（XY左下からのオフセット）
    if scalebar_px_native is not None:
        scalebar_len = max(1, int(round(scalebar_px_native * sx)))
        xy_w_res = int(round(xy_w * sx))
        xy_h_res = int(round(xy_h * sy))
        sbx = max(0, scalebar_offset[0])
        sby = max(0, xy_h_res - scalebar_offset[1])
        sbx2 = min(w-1, sbx + scalebar_len)
        sby2 = min(h-1, sby + scalebar_height)
        cv2.rectangle(tile_bgr, (sbx, sby), (sbx2, sby2), line_color, -1)

    # === パディング＋外枠
    padded = cv2.copyMakeBorder(tile_bgr, pad_px, pad_px, pad_px, pad_px,
                                cv2.BORDER_CONSTANT, value=(0,0,0))
    ph, pw = padded.shape[:2]
    cv2.rectangle(padded, (0, 0), (pw-1, ph-1), line_color, border_thickness)

    return padded


# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")
timestamps = df['Timestamp_T=0'].values

# 存在チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        print(f"{'✅' if cap.isOpened() else '❌'} 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === 基本サイズ ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio:.4f}")

# === FPS・出力 ===
fps_isop = 50.948
n_frames_isop = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60

# === レイアウト定義（ネイティブ） ===
# SPIN 3面図（603x603）
SPIN_ORIG_W, SPIN_ORIG_H = 603, 603
SPIN_XY_W, SPIN_XY_H = 512, 512
SPIN_GAP = 3
SPIN_SCALEBAR_NATIVE = 46.51  # px

# ISOP 3面図（601x401）
ISOP_ORIG_W, ISOP_ORIG_H = 601, 401
ISOP_XY_W, ISOP_XY_H = 500, 300
ISOP_GAP = 3
ISOP_SCALEBAR_NATIVE = 48.84  # px

# === 共通パラメータ ===
pad_px = 2
line_color = (255, 255, 255)

out = None

# === 時間補正とキャッシュ（SPIN） ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset
last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレーム読込
def read_first_or_black(cap, width, height_fallback=None):
    ret, frame = cap.read()
    if ret:
        return resize_fixed_width(frame, width)
    if height_fallback is None:
        height_fallback = width
    return np.zeros((height_fallback, width, 3), dtype=np.uint8)

initial_isop_fluo_frame  = read_first_or_black(caps["isop_fluo"], base_width)
initial_isop_track_frame = read_first_or_black(caps["isop_track"], base_width, initial_isop_fluo_frame.shape[0])
initial_spin_fluo_frame  = read_first_or_black(caps["spin_fluo"], base_width, initial_isop_fluo_frame.shape[0])
initial_spin_track_frame = read_first_or_black(caps["spin_track"], base_width, initial_isop_fluo_frame.shape[0])

last_isop_fluo_frame  = initial_isop_fluo_frame
last_isop_track_frame = initial_isop_track_frame
last_spin_fluo_frame  = initial_spin_fluo_frame
last_spin_track_frame = initial_spin_track_frame

for i in range(n_frames_isop):
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames（リサイズ） ===
    isop_frames = []
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        last_isop_fluo_frame = resize_fixed_width(frame, base_width)

    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        last_isop_track_frame = resize_fixed_width(frame, base_width)

    # ISOP：ギャップ白帯＋枠＋ラベル＋スケールバー（48.84px ネイティブ→リサイズ反映）
    
        # ISOP
    isop_fluo_anno = annotate_orthoview_tile(
        last_isop_fluo_frame.copy(),
        orig_w=ISOP_ORIG_W, orig_h=ISOP_ORIG_H,
        xy_w=ISOP_XY_W, xy_h=ISOP_XY_H, gap=ISOP_GAP,
        pad_px=pad_px, border_thickness=2, line_color=line_color,
        draw_labels=True,
        label_offsets_px={'xy': (14, -20), 'yz': (16, -20), 'xz': (14, 10)},  # ←ここで微調整
        scalebar_px_native=ISOP_SCALEBAR_NATIVE, scalebar_height=3,
        scalebar_offset=(12, 12)
    )

    # SPIN
    spin_fluo_anno = annotate_orthoview_tile(
        last_spin_fluo_frame.copy(),
        orig_w=SPIN_ORIG_W, orig_h=SPIN_ORIG_H,
        xy_w=SPIN_XY_W, xy_h=SPIN_XY_H, gap=SPIN_GAP,
        pad_px=pad_px, border_thickness=2, line_color=line_color,
        draw_labels=True,
        label_offsets_px={'xy': (14, -32), 'yz': (12, -32), 'xz': (14, 10)},  # ←ここで微調整
        scalebar_px_native=SPIN_SCALEBAR_NATIVE, scalebar_height=3,
        scalebar_offset=(10, 10)
    )

 
    # === SPIN frames（リサイズ & 同期） ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            last_spin_fluo_frame = resize_fixed_width(frame, base_width)
        last_spin_fluo_index = spin_fluo_index

    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                last_spin_track_frame = resize_fixed_width(frame, base_width)
            last_spin_track_index = spin_track_index
    # それ以降は最後のフレーム保持

    # SPIN：ギャップ白帯＋枠＋ラベル＋スケールバー（46.51px ネイティブ→リサイズ反映）
    spin_fluo_anno = annotate_orthoview_tile(
        last_spin_fluo_frame.copy(),
        orig_w=SPIN_ORIG_W, orig_h=SPIN_ORIG_H,
        xy_w=SPIN_XY_W, xy_h=SPIN_XY_H, gap=SPIN_GAP,
        pad_px=pad_px, border_thickness=2, line_color=line_color,
        draw_labels=True, scalebar_px_native=SPIN_SCALEBAR_NATIVE,
        scalebar_height=3, scalebar_offset=(10, 10)
    )
    spin_track_anno = annotate_orthoview_tile(
        last_spin_track_frame.copy(),
        orig_w=SPIN_ORIG_W, orig_h=SPIN_ORIG_H,
        xy_w=SPIN_XY_W, xy_h=SPIN_XY_H, gap=SPIN_GAP,
        pad_px=pad_px, border_thickness=2, line_color=line_color,
        draw_labels=True, scalebar_px_native=SPIN_SCALEBAR_NATIVE,
        scalebar_height=3, scalebar_offset=(10, 10)
    )

    # === 見出し・タイムスタンプ（ISOP左上タイルに）
    isop_fluo_anno = put_text_arial(isop_fluo_anno, "ISOP microscopy", (5, 5))
    spin_fluo_anno = put_text_arial(spin_fluo_anno, "Spinning disk confocal microscopy", (5, 5))
    isop_fluo_anno = put_text_arial(isop_fluo_anno, f"T = {timestamps[i]:.3f} s", (5, 35))

    # === 2x2 に並べる ===
    # 上段：ISOP2枚、下段：SPIN2枚
    max_h_top = max(isop_fluo_anno.shape[0], isop_track_anno.shape[0])
    max_h_bottom = max(spin_fluo_anno.shape[0], spin_track_anno.shape[0])

    def pad_to_height(img, H):
        return cv2.copyMakeBorder(img, 0, H - img.shape[0], 0, 0, cv2.BORDER_CONSTANT)

    top = np.hstack([
        pad_to_height(isop_fluo_anno, max_h_top),
        np.zeros((max_h_top, gap_h, 3), dtype=np.uint8),
        pad_to_height(isop_track_anno, max_h_top)
    ])
    bottom = np.hstack([
        pad_to_height(spin_fluo_anno, max_h_bottom),
        np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8),
        pad_to_height(spin_track_anno, max_h_bottom)
    ])

    combined_core = np.vstack([
        top,
        np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8),
        bottom
    ])

    combined = cv2.copyMakeBorder(
        combined_core, top_margin, bottom_margin, side_margin, side_margin,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )

    # === VideoWriter ===
    if out is None:
        H, W = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (W, H))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
if out is not None:
    out.release()

print(f"✅ 出力完了: {output_path}")


✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
Base width: 601, Resize ratio: 0.9967
処理中: フレーム 0/10 (t = 0.000 s)
✅ 出力完了: combined_output_newtimestamp.mp4


In [90]:
#NG
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)

def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 横幅を合わせるリサイズ ===
def resize_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(round(h * scale))
    resized = cv2.resize(image, (target_width, new_h))
    return resized

# === 3面図（603x603想定）用：ギャップに白帯 + パディング枠 ===
def draw_spin_gaps_and_border(tile, target_width, pad_px=2,
                              orig_size=603, gap_start=512, gap_end=514,
                              line_color=(255,255,255), border_thickness=2):
    """
    tile: リサイズ後の画像（幅=target_width）
    ギャップx=512..514, y=512..514 に白帯を引く（スケール対応）
    その後、パディングを追加し、パディング側に外枠を描く（中身に食い込まない）
    """
    h, w = tile.shape[:2]
    # リサイズ後のスケール
    sx = w / float(orig_size)
    sy = h / float(orig_size)  # 603→等倍のはずだが厳密に

    # 縦ギャップ（右）
    x1 = int(round(gap_start * sx))
    x2 = int(round(gap_end   * sx))
    x1, x2 = max(0, min(x1, w-1)), max(0, min(x2, w-1))
    if x2 < x1: x1, x2 = x2, x1
    cv2.rectangle(tile, (x1, 0), (x2, h-1), line_color, -1)

    # 横ギャップ（下）
    y1 = int(round(gap_start * sy))
    y2 = int(round(gap_end   * sy))
    y1, y2 = max(0, min(y1, h-1)), max(0, min(y2, h-1))
    if y2 < y1: y1, y2 = y2, y1
    cv2.rectangle(tile, (0, y1), (w-1, y2), line_color, -1)

    # パディング（黒）を追加して、そのパディング側に枠
    padded = cv2.copyMakeBorder(tile, pad_px, pad_px, pad_px, pad_px,
                                cv2.BORDER_CONSTANT, value=(0,0,0))
    ph, pw = padded.shape[:2]
    cv2.rectangle(padded, (0, 0), (pw-1, ph-1), line_color, border_thickness)
    return padded

# === 一般タイル用：中身は触らず、パディング枠のみ ===
def pad_and_border(tile, pad_px=2, line_color=(255,255,255), border_thickness=2):
    padded = cv2.copyMakeBorder(tile, pad_px, pad_px, pad_px, pad_px,
                                cv2.BORDER_CONSTANT, value=(0,0,0))
    ph, pw = padded.shape[:2]
    cv2.rectangle(padded, (0, 0), (pw-1, ph-1), line_color, border_thickness)
    return padded

# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")
timestamps = df['Timestamp_T=0'].values

# チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ 開けません: {path}")
        else:
            print(f"✅ 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === サイズ・出力 ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio:.4f}")

# === FPS・フレーム数取得 ===
fps_isop = 50.948
n_frames_isop = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60

out = None

# === 時間補正とキャッシュ ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset

last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレーム（リサイズ）
def read_first_or_black(cap, width, height_fallback=None):
    ret, frame = cap.read()
    if ret:
        return resize_fixed_width(frame, width)
    if height_fallback is None:
        height_fallback = width  # 正方形想定
    return np.zeros((height_fallback, width, 3), dtype=np.uint8)

initial_isop_fluo_frame  = read_first_or_black(caps["isop_fluo"], base_width)
initial_isop_track_frame = read_first_or_black(caps["isop_track"], base_width, initial_isop_fluo_frame.shape[0])
initial_spin_fluo_frame  = read_first_or_black(caps["spin_fluo"], base_width, initial_isop_fluo_frame.shape[0])
initial_spin_track_frame = read_first_or_black(caps["spin_track"], base_width, initial_isop_fluo_frame.shape[0])

last_isop_fluo_frame  = initial_isop_fluo_frame
last_isop_track_frame = initial_isop_track_frame
last_spin_fluo_frame  = initial_spin_fluo_frame
last_spin_track_frame = initial_spin_track_frame

# === パディング量と線色 ===
pad_px = 2
line_color = (255, 255, 255)

for i in range(n_frames_isop):
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames ===
    isop_frames = []
    current_isop_fluo_frame = last_isop_fluo_frame
    current_isop_track_frame = last_isop_track_frame

    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        current_isop_fluo_frame = resize_fixed_width(frame, base_width)
        last_isop_fluo_frame = current_isop_fluo_frame
    # （ISOP内部のクロスラインは描かず、見た目を壊さない／必要ならここで描く）
    isop_frames.append(pad_and_border(current_isop_fluo_frame, pad_px, line_color, 2))

    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        current_isop_track_frame = resize_fixed_width(frame, base_width)
        last_isop_track_frame = current_isop_track_frame
    isop_frames.append(pad_and_border(current_isop_track_frame, pad_px, line_color, 2))

    # === SPIN fluo frame ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)
    current_spin_fluo_frame = last_spin_fluo_frame

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            current_spin_fluo_frame = resize_fixed_width(frame, base_width)
            last_spin_fluo_frame = current_spin_fluo_frame
        last_spin_fluo_index = spin_fluo_index
    # ギャップ白帯 + パディング枠
    spin_fluo_frame = draw_spin_gaps_and_border(current_spin_fluo_frame, base_width, pad_px)

    # === SPIN track frame ===
    current_spin_track_frame = last_spin_track_frame
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                current_spin_track_frame = resize_fixed_width(frame, base_width)
                last_spin_track_frame = current_spin_track_frame
            last_spin_track_index = spin_track_index
        spin_track_frame = current_spin_track_frame
    else:
        spin_track_frame = last_spin_track_frame
    # ギャップ白帯 + パディング枠
    spin_track_frame = draw_spin_gaps_and_border(spin_track_frame, base_width, pad_px)

    # === ラベル・タイマー ===
    isop_frames[0] = put_text_arial(isop_frames[0], "ISOP microscopy", (5, 5))
    spin_fluo_frame = put_text_arial(spin_fluo_frame, "Spinning disk confocal microscopy", (5, 5))
    isop_frames[0] = put_text_arial(isop_frames[0], f"T = {timestamps[i]:.3f} s", (5, 35))

    # === 横結合（上段：ISOP2枚、下段：SPIN2枚） ===
    max_h_top = max(isop_frames[0].shape[0], isop_frames[1].shape[0])
    max_h_bottom = max(spin_fluo_frame.shape[0], spin_track_frame.shape[0])
    gap_h_bar_top = np.zeros((max_h_top, gap_h, 3), dtype=np.uint8)
    top = np.hstack([
        cv2.copyMakeBorder(isop_frames[0], 0, max_h_top - isop_frames[0].shape[0], 0, 0, cv2.BORDER_CONSTANT),
        gap_h_bar_top,
        cv2.copyMakeBorder(isop_frames[1], 0, max_h_top - isop_frames[1].shape[0], 0, 0, cv2.BORDER_CONSTANT)
    ])

    gap_h_bar_bottom = np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8)
    bottom = np.hstack([
        cv2.copyMakeBorder(spin_fluo_frame, 0, max_h_bottom - spin_fluo_frame.shape[0], 0, 0, cv2.BORDER_CONSTANT),
        gap_h_bar_bottom,
        cv2.copyMakeBorder(spin_track_frame, 0, max_h_bottom - spin_track_frame.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    ])

    gap_v_bar = np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8)
    combined_core = np.vstack((top, gap_v_bar, bottom))

    combined = cv2.copyMakeBorder(
        combined_core, top_margin, bottom_margin, side_margin, side_margin,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )

    # === 座標ラベル（位置は元コードのまま。pad分だけ僅かにズレるが実害なし） ===
    base_height = initial_isop_fluo_frame.shape[0]  # 参照用
    label_positions = {
        "xy": (side_margin + 10, top_margin + 265),
        "yz": (side_margin + 565, top_margin + 265),
        "xz": (side_margin + 10, top_margin + 300)
    }
    for text, (x, y) in label_positions.items():
        combined = put_text_arial_italic(combined, text, (x, y))
        combined = put_text_arial_italic(combined, text, (x, y + base_height + gap_v + 200))

    # === スケールバー（元コードを踏襲） ===
    # マスク領域
    cv2.rectangle(combined, (gap_h + side_margin + base_width + 435, top_margin + 275),
                  (gap_h + side_margin + base_width + 490, top_margin + 290), (0, 0, 0), -1)

    scale_bar_length = (48, int(44 * resize_ratio))  # isop_fluoのスケールバー長さ
    scale_bar_height = 3
    scale_bar_color = (255, 255, 255)
    scale_bar_position = (437, 280)
    for j in range(2):
        x = gap_h + scale_bar_position[0]
        y = top_margin + (gap_v + base_height + 200) * j + scale_bar_position[1]
        cv2.rectangle(combined, (x, y), (x + scale_bar_length[j], y + scale_bar_height),
                      scale_bar_color, -1)

    # === 初回に VideoWriter 初期化 ===
    if out is None:
        h, w = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (w, h))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
if out is not None:
    out.release()

print(f"✅ 出力完了: {output_path}")




✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
Base width: 601, Resize ratio: 0.9967
処理中: フレーム 0/10 (t = 0.000 s)
✅ 出力完了: combined_output_newtimestamp.mp4


In [1]:
#Do not use
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)


def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 横幅を合わせるリサイズ ===
def resize_and_add_crosshair_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(h * scale)
    resized = cv2.resize(image, (target_width, new_h))
    return resized

# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"  # ← 追加：CSVのパス

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")

timestamps = df['Timestamp_T=0'].values  # NumPy配列に変換

# チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ 開けません: {path}")
        else:
            print(f"✅ 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === サイズ・出力 ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio}")

# === FPS・フレーム数取得 === 
# コメントアウトでFPSを変更;; 50.948:終了時刻から算出　50.943:各フレームの時間差平均から算出
fps_isop = 50.948  
#fps_isop = 50.943
n_frames_isop = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60
base_height = sizes["isop_fluo"][1] # base_heightをisop_fluoの高さに設定

out = None  # 動的作成

# === 時間補正とキャッシュ ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset

last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレームを読み込み、またはダミーフレームを作成
# これにより、最初のフレームが黒くなるのを防ぎます
ret, initial_isop_fluo_frame = caps["isop_fluo"].read()
if ret:
    last_isop_fluo_frame = resize_and_add_crosshair_fixed_width(initial_isop_fluo_frame, base_width)
else:
    last_isop_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8) # 適切なサイズで初期化

ret, initial_isop_track_frame = caps["isop_track"].read()
if ret:
    last_isop_track_frame = resize_and_add_crosshair_fixed_width(initial_isop_track_frame, base_width)
else:
    last_isop_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_fluo_frame = caps["spin_fluo"].read()
if ret:
    last_spin_fluo_frame = resize_and_add_crosshair_fixed_width(initial_spin_fluo_frame, base_width)
else:
    last_spin_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_track_frame = caps["spin_track"].read()
if ret:
    last_spin_track_frame = resize_and_add_crosshair_fixed_width(initial_spin_track_frame, base_width)
else:
    last_spin_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)


for i in range(n_frames_isop): # 処理フレーム数を必要に応じて調整
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames ===
    isop_frames = []
    current_isop_fluo_frame = last_isop_fluo_frame # デフォルトを前のフレームに設定
    current_isop_track_frame = last_isop_track_frame

    # isop_fluo
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        current_isop_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_fluo_frame = current_isop_fluo_frame # 最新のフレームを更新
    isop_frames.append(current_isop_fluo_frame)

    # isop_track
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        current_isop_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_track_frame = current_isop_track_frame # 最新のフレームを更新
    isop_frames.append(current_isop_track_frame)

    # === SPIN fluo frame ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)
    current_spin_fluo_frame = last_spin_fluo_frame # デフォルトを前のフレームに設定

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            current_spin_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
            last_spin_fluo_frame = current_spin_fluo_frame # 最新のフレームを更新
        last_spin_fluo_index = spin_fluo_index
    spin_fluo_frame = current_spin_fluo_frame

    # === SPIN track frame ===
    current_spin_track_frame = last_spin_track_frame # デフォルトを前のフレームに設定
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                current_spin_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
                last_spin_track_frame = current_spin_track_frame # 最新のフレームを更新
            last_spin_track_index = spin_track_index
        spin_track_frame = current_spin_track_frame
    else:
        # 終了時間後に新しいフレームがない場合は、最後の有効なフレームを使用し続ける
        spin_track_frame = last_spin_track_frame


    # クロスライン・外枠
    line_color = (255, 255, 255)
    thickness = -1
    for frame in [isop_frames[0], isop_frames[1], spin_fluo_frame, spin_track_frame]:
        if frame is isop_frames[0] or frame is isop_frames[1]:
            cv2.rectangle(frame, (501, 0), (503, 401), line_color, thickness)
            cv2.rectangle(frame, (0, 301), (601, 303), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (600, 302), line_color, 2)
            cv2.rectangle(frame, (0, 0), (502, 402), line_color, 2)
        else:
            cv2.rectangle(frame, (501, 0), (503, 601), line_color, thickness)
            cv2.rectangle(frame, (0, 500), (601, 503), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (600, 501), line_color, 2)
            cv2.rectangle(frame, (0, 0), (500, 601), line_color, 2)


    # ラベル・タイマー
    isop_frames[0] = put_text_arial(isop_frames[0], "ISOP microscopy", (5, 5))
    spin_fluo_frame = put_text_arial(spin_fluo_frame, "Spinning disk confocal microscopy", (5, 5))
    isop_frames[0] = put_text_arial(isop_frames[0], f"T = {timestamps[i]:.3f} s", (5, 35))


    # === 横結合 ===
    max_h_top = max(isop_frames[0].shape[0], isop_frames[1].shape[0])
    max_h_bottom = max(spin_fluo_frame.shape[0], spin_track_frame.shape[0])
    gap_h_bar = np.zeros((max_h_top, gap_h, 3), dtype=np.uint8)
    top = np.hstack([cv2.copyMakeBorder(f, 0, max_h_top - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in isop_frames])
    top = np.hstack((top[:, :base_width], gap_h_bar, top[:, base_width:]))

    gap_h_bar2 = np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8)
    bottom = np.hstack([cv2.copyMakeBorder(f, 0, max_h_bottom - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in [spin_fluo_frame, spin_track_frame]])
    bottom = np.hstack((bottom[:, :base_width], gap_h_bar2, bottom[:, base_width:]))

    gap_v_bar = np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8)
    combined_core = np.vstack((top, gap_v_bar, bottom))

    combined = cv2.copyMakeBorder(combined_core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # 下線の補正
    cv2.rectangle(combined, (side_margin, 400 + top_margin), (503 + side_margin, 402 + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 400 + top_margin), (1104 + gap_h + side_margin, 402 + top_margin), line_color , thickness)
    cv2.rectangle(combined, (side_margin, 1001 + gap_v + top_margin), (503 + side_margin, 1003 + gap_v + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 1001 + gap_v + top_margin), (1104 + gap_h + side_margin, 1003 + gap_v + top_margin), line_color , thickness)    

    # 座標ラベル
    label_positions = {
        "xy": (side_margin + 10, top_margin + 265),
        "yz": (side_margin + 565, top_margin + 265),
        "xz": (side_margin + 10, top_margin + 300)
    }

    # スケールバーマスク
    cv2.rectangle(combined, (gap_h + side_margin + base_width + 435, top_margin + 275), 
                  (gap_h + side_margin + base_width + 490, top_margin + 290), (0, 0, 0), -1)

    for text, (x, y) in label_positions.items():
        combined = put_text_arial_italic(combined, text, (x, y))
        combined = put_text_arial_italic(combined, text, (x, y + base_height + gap_v + 200))

    # スケールバー
    scale_bar_length = (48, int(44 * resize_ratio))  # isop_fluoのスケールバー長さ
    scale_bar_height = 3
    scale_bar_color = (255, 255, 255)
    scale_bar_position = (437, 280)

    for i in range(2):
        x = gap_h + scale_bar_position[0]
        y = top_margin + (gap_v + base_height + 200) * i + scale_bar_position[1]
        cv2.rectangle(combined, (x, y), (x + scale_bar_length[i], y + scale_bar_height), scale_bar_color, -1)
    

    # 初回に VideoWriter 初期化
    if out is None:
        h, w = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (w, h))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
out.release()

print(f"✅ 出力完了: {output_path}")

✅ 開けました: isop_fluo2.avi
✅ 開けました: isop_track-2.avi
✅ 開けました: spin-fluo2.avi
✅ 開けました: spin_track-3.avi
Base width: 601, Resize ratio: 0.9693548387096774
処理中: フレーム 0/9996 (t = 0.000 s)
処理中: フレーム 50/9996 (t = 0.981 s)
処理中: フレーム 100/9996 (t = 1.963 s)
処理中: フレーム 150/9996 (t = 2.944 s)
処理中: フレーム 200/9996 (t = 3.926 s)
処理中: フレーム 250/9996 (t = 4.907 s)
処理中: フレーム 300/9996 (t = 5.888 s)
処理中: フレーム 350/9996 (t = 6.870 s)
処理中: フレーム 400/9996 (t = 7.851 s)
処理中: フレーム 450/9996 (t = 8.833 s)
処理中: フレーム 500/9996 (t = 9.814 s)
処理中: フレーム 550/9996 (t = 10.795 s)
処理中: フレーム 600/9996 (t = 11.777 s)
処理中: フレーム 650/9996 (t = 12.758 s)
処理中: フレーム 700/9996 (t = 13.739 s)
処理中: フレーム 750/9996 (t = 14.721 s)
処理中: フレーム 800/9996 (t = 15.702 s)
処理中: フレーム 850/9996 (t = 16.684 s)
処理中: フレーム 900/9996 (t = 17.665 s)
処理中: フレーム 950/9996 (t = 18.646 s)
処理中: フレーム 1000/9996 (t = 19.628 s)
処理中: フレーム 1050/9996 (t = 20.609 s)
処理中: フレーム 1100/9996 (t = 21.591 s)
処理中: フレーム 1150/9996 (t = 22.572 s)
処理中: フレーム 1200/9996 (t = 23.553 s)
処理中: フレーム

In [ ]:
##レイアウトOK##

import cv2
import numpy as np
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font = ImageFont.truetype(font_path, 25)
font_I = ImageFont.truetype(font_path_I, 30)

def put_text(img, text, position, font, color=(255,255,255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 共通パラメータ ===
pad = 40
border_thickness = 2
line_color = (255, 255, 255)

# === パス設定 ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"
df = pd.read_csv(timestamp_csv_path)
timestamps = df["Timestamp_T=0"].values

# === フレーム取得 & パディング ===
def read_frame_and_pad(path, target_width):
    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return np.zeros((600, target_width, 3), dtype=np.uint8)
    h, w = frame.shape[:2]
    scale = target_width / w
    frame = cv2.resize(frame, (target_width, int(h * scale)))
    return cv2.copyMakeBorder(frame, pad, pad, pad, pad, cv2.BORDER_CONSTANT)

base_width = 600
frames = {k: read_frame_and_pad(v, base_width) for k, v in paths.items()}

# === クロスライン描画 ===
def draw_cross_lines(frame, xy_type):
    if xy_type == "isop":
        cv2.rectangle(frame, (pad + 501, pad), (pad + 503, pad + 401), line_color, -1)
        cv2.rectangle(frame, (pad, pad + 301), (pad + 599, pad + 303), line_color, -1)
    elif xy_type == "spin":
        cv2.rectangle(frame, (pad + 513, pad), (pad + 515, pad + 599), line_color, -1)
        cv2.rectangle(frame, (pad, pad + 513), (pad + 599, pad + 515), line_color, -1)

# === 修正済み外枠描画 ===
def draw_border(frame, xy_type):
    if xy_type == "isop":
        # 枠を途中で止める（右下は空ける）
        cv2.line(frame, (pad, pad), (pad + 599, pad), line_color, border_thickness)        # 上
        cv2.line(frame, (pad, pad), (pad, pad + 401), line_color, border_thickness)        # 左
        cv2.line(frame, (pad, pad + 401), (pad + 501, pad + 401), line_color, border_thickness)  # 下
        cv2.line(frame, (pad + 599, pad), (pad + 599, pad + 301), line_color, border_thickness)  # 右

    elif xy_type == "spin":
        cv2.line(frame, (pad, pad), (pad + 599, pad), line_color, border_thickness)        # 上
        cv2.line(frame, (pad, pad), (pad, pad + 599), line_color, border_thickness)        # 左
        cv2.line(frame, (pad, pad + 599), (pad + 513, pad + 599), line_color, border_thickness)  # 下
        cv2.line(frame, (pad + 599, pad), (pad + 599, pad + 513), line_color, border_thickness)  # 右

# === 描画実行 ===
for key in frames:
    kind = "isop" if "isop" in key else "spin"
    draw_cross_lines(frames[key], kind)
    draw_border(frames[key], kind)

# === ラベル・スケールバー ===
frames["isop_fluo"] = put_text(frames["isop_fluo"], "ISOP microscopy", (pad + 5, pad + 5), font)
frames["isop_fluo"] = put_text(frames["isop_fluo"], f"T = {timestamps[0]:.3f} s", (pad + 5, pad + 35), font)
frames["spin_fluo"] = put_text(frames["spin_fluo"], "Spinning disk confocal microscopy", (pad + 5, pad + 5), font)

# === 上下結合 ===
gap_h, gap_v = 15, 40
side_margin, top_margin, bottom_margin = 20, 60, 60

def hstack_with_gap(img1, img2, gap=gap_h):
    h = max(img1.shape[0], img2.shape[0])
    g = np.zeros((h, gap, 3), dtype=np.uint8)
    img1 = cv2.copyMakeBorder(img1, 0, h - img1.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    img2 = cv2.copyMakeBorder(img2, 0, h - img2.shape[0], 0, 0, cv2.BORDER_CONSTANT)
    return np.hstack([img1, g, img2])

top = hstack_with_gap(frames["isop_fluo"], frames["isop_track"])
bottom = hstack_with_gap(frames["spin_fluo"], frames["spin_track"])
core = np.vstack([top, np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8), bottom])
combined = cv2.copyMakeBorder(core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT)

# === ラベル位置 ===
def draw_labels_on_combined(combined_img):
    combined_img = put_text(combined_img, "xy", (side_margin + pad + 10, top_margin + pad + 265), font_I)
    combined_img = put_text(combined_img, "zy", (side_margin + pad + 560, top_margin + pad + 265), font_I)
    combined_img = put_text(combined_img, "xz", (side_margin + pad + 10, top_margin + pad + 300), font_I)
    offset_y = top.shape[0] + gap_v
    combined_img = put_text(combined_img, "xy", (side_margin + pad + 10, top_margin + offset_y + pad + 477), font_I)
    combined_img = put_text(combined_img, "zy", (side_margin + pad + 560, top_margin + offset_y + pad + 477), font_I)
    combined_img = put_text(combined_img, "xz", (side_margin + pad + 10, top_margin + offset_y + pad + 510), font_I)
    return combined_img

combined = draw_labels_on_combined(combined)

# === スケールバー ===
def draw_scale_bar(img, top_pos, length):
    x, y = top_pos
    cv2.rectangle(img, (x, y), (x + length, y + 3), (255, 255, 255), -1)

draw_scale_bar(combined, (side_margin + gap_h + pad + 420, top_margin + pad + 280), 48)
draw_scale_bar(combined, (side_margin + gap_h + pad + 437, top_margin + top.shape[0] + gap_v + pad + 490), 44)

# === 保存 ===
cv2.imwrite("combined_frame_cleaned_fixed.png", combined)
print("✅ 完了: combined_frame_cleaned_fixed.png")


In [23]:
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)


def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 横幅を合わせるリサイズ ===
def resize_and_add_crosshair_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(h * scale)
    resized = cv2.resize(image, (target_width, new_h))
    return resized

# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"  # ← 追加：CSVのパス

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")

timestamps = df['Timestamp_T=0'].values  # NumPy配列に変換

# チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ 開けません: {path}")
        else:
            print(f"✅ 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === サイズ・出力 ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio}")

# === FPS・フレーム数取得 === 
# コメントアウトでFPSを変更;; 50.948:終了時刻から算出　50.943:各フレームの時間差平均から算出
fps_isop = 50.948  
#fps_isop = 50.943
n_frames_isop = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60
base_height = sizes["isop_fluo"][1] # base_heightをisop_fluoの高さに設定

out = None  # 動的作成

# === 時間補正とキャッシュ ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset

last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレームを読み込み、またはダミーフレームを作成
# これにより、最初のフレームが黒くなるのを防ぎます
ret, initial_isop_fluo_frame = caps["isop_fluo"].read()
if ret:
    last_isop_fluo_frame = resize_and_add_crosshair_fixed_width(initial_isop_fluo_frame, base_width)
else:
    last_isop_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8) # 適切なサイズで初期化

ret, initial_isop_track_frame = caps["isop_track"].read()
if ret:
    last_isop_track_frame = resize_and_add_crosshair_fixed_width(initial_isop_track_frame, base_width)
else:
    last_isop_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_fluo_frame = caps["spin_fluo"].read()
if ret:
    last_spin_fluo_frame = resize_and_add_crosshair_fixed_width(initial_spin_fluo_frame, base_width)
else:
    last_spin_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_track_frame = caps["spin_track"].read()
if ret:
    last_spin_track_frame = resize_and_add_crosshair_fixed_width(initial_spin_track_frame, base_width)
else:
    last_spin_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)


for i in range(n_frames_isop): # 処理フレーム数を必要に応じて調整
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames ===
    isop_frames = []
    current_isop_fluo_frame = last_isop_fluo_frame # デフォルトを前のフレームに設定
    current_isop_track_frame = last_isop_track_frame

    # isop_fluo
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        current_isop_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_fluo_frame = current_isop_fluo_frame # 最新のフレームを更新
    isop_frames.append(current_isop_fluo_frame)

    # isop_track
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        current_isop_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_track_frame = current_isop_track_frame # 最新のフレームを更新
    isop_frames.append(current_isop_track_frame)

    # === SPIN fluo frame ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)
    current_spin_fluo_frame = last_spin_fluo_frame # デフォルトを前のフレームに設定

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            current_spin_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
            last_spin_fluo_frame = current_spin_fluo_frame # 最新のフレームを更新
        last_spin_fluo_index = spin_fluo_index
    spin_fluo_frame = current_spin_fluo_frame

    # === SPIN track frame ===
    current_spin_track_frame = last_spin_track_frame # デフォルトを前のフレームに設定
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                current_spin_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
                last_spin_track_frame = current_spin_track_frame # 最新のフレームを更新
            last_spin_track_index = spin_track_index
        spin_track_frame = current_spin_track_frame
    else:
        # 終了時間後に新しいフレームがない場合は、最後の有効なフレームを使用し続ける
        spin_track_frame = last_spin_track_frame

    # クロスライン・外枠
    line_color = (255, 255, 255)
    thickness = 1

    for frame in [isop_frames[0], isop_frames[1], spin_fluo_frame, spin_track_frame]:
        if frame is isop_frames[0] or frame is isop_frames[1]:
            # ISOP領域（従来通り）
            cv2.rectangle(frame, (501, 0), (503, 401), line_color, thickness)
            cv2.rectangle(frame, (0, 301), (601, 303), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (600, 302), line_color, 2)
            cv2.rectangle(frame, (0, 0), (502, 402), line_color, 2)
        #else:
            # SPIN領域（新サイズ 603×603 に対応）
            #cv2.rectangle(frame, (513, 0), (515, 603), line_color, thickness)        # 中央縦線
            #cv2.rectangle(frame, (0, 513), (603, 515), line_color, thickness)        # 中央横線
            #cv2.line(frame, (0, 0), (604, 0), line_color, 2)                          # 上辺
            #cv2.line(frame, (0, 0), (0, 604), line_color, 2)                          # 左辺
            #cv2.line(frame, (0, 604), (515, 604), line_color, 2)                      # 下辺（右下開ける）
            #cv2.line(frame, (604, 0), (604, 515), line_color, 2)                      # 右辺（右下開ける）

            
        #    cv2.rectangle(frame, (513, 0), (513, 604), line_color, thickness)
          #  cv2.rectangle(frame, (0, 513), (604, 513), line_color, thickness)
            #cv2.rectangle(frame, (0, 0), (604, 513), line_color, 2)
            #cv2.rectangle(frame, (0, 0), (513, 604), line_color, 2)

            
        else:
        # SPIN領域（新サイズ 603×603 に対応）

            # クロスライン
            cv2.line(frame, (513, 0), (515, 603), line_color, thickness)  # 中央縦線（x = 603 // 2）
            cv2.line(frame, (0, 513), (603, 515), line_color, thickness)  # 中央横線（y = 603 // 2）

            # 外枠（完全な長方形）
            cv2.rectangle(frame, (0, 0), (602, 602), line_color, 2)  # (終点は inclusive なので -1)

            # 右枠線（必要であれば明示的に重ね書き）
            cv2.line(frame, (603, 0), (603, 603), line_color, 2)

            # 下枠線と交わるよう調整
            cv2.line(frame, (0, 603), (603, 603), line_color, 2)


            
    # ラベル・タイマー
    isop_frames[0] = put_text_arial(isop_frames[0], "ISOP microscopy", (5, 5))
    spin_fluo_frame = put_text_arial(spin_fluo_frame, "Spinning disk confocal microscopy", (5, 5))
    isop_frames[0] = put_text_arial(isop_frames[0], f"T = {timestamps[i]:.3f} s", (5, 35))


    # === 横結合 ===
    max_h_top = max(isop_frames[0].shape[0], isop_frames[1].shape[0])
    max_h_bottom = max(spin_fluo_frame.shape[0], spin_track_frame.shape[0])
    gap_h_bar = np.zeros((max_h_top, gap_h, 3), dtype=np.uint8)
    top = np.hstack([cv2.copyMakeBorder(f, 0, max_h_top - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in isop_frames])
    top = np.hstack((top[:, :base_width], gap_h_bar, top[:, base_width:]))

    gap_h_bar2 = np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8)
    bottom = np.hstack([cv2.copyMakeBorder(f, 0, max_h_bottom - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in [spin_fluo_frame, spin_track_frame]])
    bottom = np.hstack((bottom[:, :base_width], gap_h_bar2, bottom[:, base_width:]))

    gap_v_bar = np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8)
    combined_core = np.vstack((top, gap_v_bar, bottom))

    combined = cv2.copyMakeBorder(combined_core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # 下線の補正
    cv2.rectangle(combined, (side_margin, 400 + top_margin), (503 + side_margin, 402 + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 400 + top_margin), (1104 + gap_h + side_margin, 402 + top_margin), line_color , thickness)
    cv2.rectangle(combined, (side_margin, 1001 + gap_v + top_margin), (503 + side_margin, 1003 + gap_v + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 1001 + gap_v + top_margin), (1104 + gap_h + side_margin, 1003 + gap_v + top_margin), line_color , thickness)    

    # 座標ラベル
    label_positions = {
        "xy": (side_margin + 10, top_margin + 265),
        "zy": (side_margin + 565, top_margin + 265),
        "xz": (side_margin + 10, top_margin + 300)
    }

    # スケールバーマスク
    cv2.rectangle(combined, (gap_h + side_margin + base_width + 435, top_margin + 275), 
                  (gap_h + side_margin + base_width + 490, top_margin + 290), (0, 0, 0), -1)

    for text, (x, y) in label_positions.items():
        combined = put_text_arial_italic(combined, text, (x, y))
        combined = put_text_arial_italic(combined, text, (x, y + base_height + gap_v + 200))

    # スケールバー
    scale_bar_length = (48, int(44 * resize_ratio))  # isop_fluoのスケールバー長さ
    scale_bar_height = 3
    scale_bar_color = (255, 255, 255)
    scale_bar_position = (437, 280)

    for i in range(2):
        x = gap_h + scale_bar_position[0]
        y = top_margin + (gap_v + base_height + 200) * i + scale_bar_position[1]
        cv2.rectangle(combined, (x, y), (x + scale_bar_length[i], y + scale_bar_height), scale_bar_color, -1)
    

    # 初回に VideoWriter 初期化
    if out is None:
        h, w = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (w, h))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
out.release()

print(f"✅ 出力完了: {output_path}")

✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
Base width: 601, Resize ratio: 0.9966832504145937
処理中: フレーム 0/10 (t = 0.000 s)
✅ 出力完了: combined_output_newtimestamp.mp4


In [39]:
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image

# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)


def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 横幅を合わせるリサイズ ===
def resize_and_add_crosshair_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(h * scale)
    resized = cv2.resize(image, (target_width, new_h))
    return resized

# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"  # ← 追加：CSVのパス

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")

timestamps = df['Timestamp_T=0'].values  # NumPy配列に変換

# チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ 開けません: {path}")
        else:
            print(f"✅ 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === サイズ・出力 ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio}")

# === FPS・フレーム数取得 === 
# コメントアウトでFPSを変更;; 50.948:終了時刻から算出　50.943:各フレームの時間差平均から算出
fps_isop = 50.948  
#fps_isop = 50.943
n_frames_isop = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60
base_height = sizes["isop_fluo"][1] # base_heightをisop_fluoの高さに設定

out = None  # 動的作成

# === 時間補正とキャッシュ ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset

last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレームを読み込み、またはダミーフレームを作成
# これにより、最初のフレームが黒くなるのを防ぎます
ret, initial_isop_fluo_frame = caps["isop_fluo"].read()
if ret:
    last_isop_fluo_frame = resize_and_add_crosshair_fixed_width(initial_isop_fluo_frame, base_width)
else:
    last_isop_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8) # 適切なサイズで初期化

ret, initial_isop_track_frame = caps["isop_track"].read()
if ret:
    last_isop_track_frame = resize_and_add_crosshair_fixed_width(initial_isop_track_frame, base_width)
else:
    last_isop_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_fluo_frame = caps["spin_fluo"].read()
if ret:
    last_spin_fluo_frame = resize_and_add_crosshair_fixed_width(initial_spin_fluo_frame, base_width)
else:
    last_spin_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_track_frame = caps["spin_track"].read()
if ret:
    last_spin_track_frame = resize_and_add_crosshair_fixed_width(initial_spin_track_frame, base_width)
else:
    last_spin_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)


for i in range(n_frames_isop): # 処理フレーム数を必要に応じて調整
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames ===
    isop_frames = []
    current_isop_fluo_frame = last_isop_fluo_frame # デフォルトを前のフレームに設定
    current_isop_track_frame = last_isop_track_frame

    # isop_fluo
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        current_isop_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_fluo_frame = current_isop_fluo_frame # 最新のフレームを更新
    isop_frames.append(current_isop_fluo_frame)

    # isop_track
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        current_isop_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_track_frame = current_isop_track_frame # 最新のフレームを更新
    isop_frames.append(current_isop_track_frame)

    # === SPIN fluo frame ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)
    current_spin_fluo_frame = last_spin_fluo_frame # デフォルトを前のフレームに設定

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            current_spin_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
            last_spin_fluo_frame = current_spin_fluo_frame # 最新のフレームを更新
        last_spin_fluo_index = spin_fluo_index
    spin_fluo_frame = current_spin_fluo_frame

    # === SPIN track frame ===
    current_spin_track_frame = last_spin_track_frame # デフォルトを前のフレームに設定
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                current_spin_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
                last_spin_track_frame = current_spin_track_frame # 最新のフレームを更新
            last_spin_track_index = spin_track_index
        spin_track_frame = current_spin_track_frame
    else:
        # 終了時間後に新しいフレームがない場合は、最後の有効なフレームを使用し続ける
        spin_track_frame = last_spin_track_frame

   # クロスライン・外枠
    line_color = (255, 255, 255)
    thickness = -1
    for frame in [isop_frames[0], isop_frames[1], spin_fluo_frame, spin_track_frame]:
        if frame is isop_frames[0] or frame is isop_frames[1]:
            cv2.rectangle(frame, (501, 0), (503, 401), line_color, thickness)
            cv2.rectangle(frame, (0, 301), (601, 303), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (600, 302), line_color, 2)
            cv2.rectangle(frame, (0, 0), (502, 402), line_color, 2)
        else:
            cv2.rectangle(frame, (513, 0), (513, 604), line_color, thickness)
            cv2.rectangle(frame, (0, 513), (604, 513), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (604, 513), line_color, 2)
            cv2.rectangle(frame, (0, 0), (513, 604), line_color, 2)
            cv2.line(frame, (602, 0), (602, 602), line_color, 2)

            
    # ラベル・タイマー
    isop_frames[0] = put_text_arial(isop_frames[0], "ISOP microscopy", (5, 5))
    spin_fluo_frame = put_text_arial(spin_fluo_frame, "Spinning disk confocal microscopy", (5, 5))
    isop_frames[0] = put_text_arial(isop_frames[0], f"T = {timestamps[i]:.3f} s", (5, 35))


    # === 横結合 ===
    max_h_top = max(isop_frames[0].shape[0], isop_frames[1].shape[0])
    max_h_bottom = max(spin_fluo_frame.shape[0], spin_track_frame.shape[0])
    gap_h_bar = np.zeros((max_h_top, gap_h, 3), dtype=np.uint8)
    top = np.hstack([cv2.copyMakeBorder(f, 0, max_h_top - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in isop_frames])
    top = np.hstack((top[:, :base_width], gap_h_bar, top[:, base_width:]))

    gap_h_bar2 = np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8)
    bottom = np.hstack([cv2.copyMakeBorder(f, 0, max_h_bottom - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in [spin_fluo_frame, spin_track_frame]])
    bottom = np.hstack((bottom[:, :base_width], gap_h_bar2, bottom[:, base_width:]))

    gap_v_bar = np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8)
    combined_core = np.vstack((top, gap_v_bar, bottom))

    combined = cv2.copyMakeBorder(combined_core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # 下線の補正
    cv2.rectangle(combined, (side_margin, 400 + top_margin), (503 + side_margin, 402 + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 400 + top_margin), (1104 + gap_h + side_margin, 402 + top_margin), line_color , thickness)
    cv2.rectangle(combined, (side_margin, 1001 + gap_v + top_margin), (513 + side_margin, 1003 + gap_v + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 1001 + gap_v + top_margin), (1114 + gap_h + side_margin, 1003 + gap_v + top_margin), line_color , thickness)    

    # 座標ラベル
    label_positions = {
        "xy": (side_margin + 10, top_margin + 265),
        "zy": (side_margin + 565, top_margin + 265),
        "xz": (side_margin + 10, top_margin + 300)
    }

    # スケールバーマスク
    cv2.rectangle(combined, (gap_h + side_margin + base_width + 435, top_margin + 275), 
                  (gap_h + side_margin + base_width + 490, top_margin + 290), (0, 0, 0), -1)

    for text, (x, y) in label_positions.items():
        combined = put_text_arial_italic(combined, text, (x, y))
        combined = put_text_arial_italic(combined, text, (x, y + base_height + gap_v + 200))

    # スケールバー
    scale_bar_length = (48, int(44 * resize_ratio))  # isop_fluoのスケールバー長さ
    scale_bar_height = 3
    scale_bar_color = (255, 255, 255)
    scale_bar_position = (437, 280)

    for i in range(2):
        x = gap_h + scale_bar_position[0]
        y = top_margin + (gap_v + base_height + 200) * i + scale_bar_position[1]
        cv2.rectangle(combined, (x, y), (x + scale_bar_length[i], y + scale_bar_height), scale_bar_color, -1)
    

    # 初回に VideoWriter 初期化
    if out is None:
        h, w = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (w, h))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
out.release()

print(f"✅ 出力完了: {output_path}")

✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
Base width: 601, Resize ratio: 0.9966832504145937
処理中: フレーム 0/10 (t = 0.000 s)
✅ 出力完了: combined_output_newtimestamp.mp4


In [66]:
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image


# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)


def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 横幅を合わせるリサイズ ===
def resize_and_add_crosshair_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(h * scale)
    resized = cv2.resize(image, (target_width, new_h))
    return resized


def put_text_pil(img, text, position, font, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)


# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"  # ← 追加：CSVのパス

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")

timestamps = df['Timestamp_T=0'].values  # NumPy配列に変換

# チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ 開けません: {path}")
        else:
            print(f"✅ 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === サイズ・出力 ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio}")

# === FPS・フレーム数取得 === 
# コメントアウトでFPSを変更;; 50.948:終了時刻から算出　50.943:各フレームの時間差平均から算出
fps_isop = 50.948  
#fps_isop = 50.943
n_frames_isop = 10
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60
base_height = sizes["isop_fluo"][1] # base_heightをisop_fluoの高さに設定

# === ラベル位置の定義をここに追加 ===
label_positions_isop = {
    "xy": (side_margin-10, top_margin+200),
    "zy": (side_margin+540, top_margin + 200),
    "xz": (side_margin-10, top_margin+240)
}
label_positions_spin = {
    "xy": (side_margin-10, top_margin +410),
    "zy": (side_margin + 540, top_margin +410),
    "xz": (side_margin-10, top_margin+ 450)
}


out = None  # 動的作成

# === 時間補正とキャッシュ ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset

last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレームを読み込み、またはダミーフレームを作成
# これにより、最初のフレームが黒くなるのを防ぎます
ret, initial_isop_fluo_frame = caps["isop_fluo"].read()
if ret:
    last_isop_fluo_frame = resize_and_add_crosshair_fixed_width(initial_isop_fluo_frame, base_width)
else:
    last_isop_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8) # 適切なサイズで初期化

ret, initial_isop_track_frame = caps["isop_track"].read()
if ret:
    last_isop_track_frame = resize_and_add_crosshair_fixed_width(initial_isop_track_frame, base_width)
else:
    last_isop_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_fluo_frame = caps["spin_fluo"].read()
if ret:
    last_spin_fluo_frame = resize_and_add_crosshair_fixed_width(initial_spin_fluo_frame, base_width)
else:
    last_spin_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_track_frame = caps["spin_track"].read()
if ret:
    last_spin_track_frame = resize_and_add_crosshair_fixed_width(initial_spin_track_frame, base_width)
else:
    last_spin_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)


for i in range(n_frames_isop): # 処理フレーム数を必要に応じて調整
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames ===
    isop_frames = []
    current_isop_fluo_frame = last_isop_fluo_frame # デフォルトを前のフレームに設定
    current_isop_track_frame = last_isop_track_frame

    # isop_fluo
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        current_isop_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_fluo_frame = current_isop_fluo_frame # 最新のフレームを更新
    isop_frames.append(current_isop_fluo_frame)

    # isop_track
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        current_isop_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_track_frame = current_isop_track_frame # 最新のフレームを更新
    isop_frames.append(current_isop_track_frame)

    # === SPIN fluo frame ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)
    current_spin_fluo_frame = last_spin_fluo_frame # デフォルトを前のフレームに設定

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            current_spin_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
            last_spin_fluo_frame = current_spin_fluo_frame # 最新のフレームを更新
        last_spin_fluo_index = spin_fluo_index
    spin_fluo_frame = current_spin_fluo_frame

    # === SPIN track frame ===
    current_spin_track_frame = last_spin_track_frame # デフォルトを前のフレームに設定
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                current_spin_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
                last_spin_track_frame = current_spin_track_frame # 最新のフレームを更新
            last_spin_track_index = spin_track_index
        spin_track_frame = current_spin_track_frame
    else:
        # 終了時間後に新しいフレームがない場合は、最後の有効なフレームを使用し続ける
        spin_track_frame = last_spin_track_frame

      # クロスライン・外枠
    line_color = (255, 255, 255)
    thickness = -1
    for frame in [isop_frames[0], isop_frames[1], spin_fluo_frame, spin_track_frame]:
        if frame is isop_frames[0] or frame is isop_frames[1]:
            cv2.rectangle(frame, (501, 0), (503, 401), line_color, thickness)
            cv2.rectangle(frame, (0, 301), (601, 303), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (600, 302), line_color, 2)
            cv2.rectangle(frame, (0, 0), (502, 402), line_color, 2)
        else:
            h, w = frame.shape[:2]  # このフレームのサイズに合わせる
            cv2.rectangle(frame, (513, 0), (513, 604), line_color, thickness)
            cv2.rectangle(frame, (0, 513), (604, 513), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (604, 513), line_color, 2)
            cv2.rectangle(frame, (0, 0), (513, 604), line_color, 2)
            cv2.line(frame, (w - 1, 0), (w - 1, 513), line_color, 2)  # 右枠線 ← 修正点

            
    # ラベル・タイマー
    isop_frames[0] = put_text_arial(isop_frames[0], "ISOP microscopy", (5, 5))
    spin_fluo_frame = put_text_arial(spin_fluo_frame, "Spinning disk confocal microscopy", (5, 5))
    isop_frames[0] = put_text_arial(isop_frames[0], f"T = {timestamps[i]:.3f} s", (5, 35))
    
    # xyzラベル
    isop_frames[0] = put_text_pil(isop_frames[0], "xy", label_positions_isop["xy"], font_I)
    isop_frames[0] = put_text_pil(isop_frames[0], "zy", label_positions_isop["zy"], font_I)
    isop_frames[0] = put_text_pil(isop_frames[0], "xz", label_positions_isop["xz"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "xy", label_positions_spin["xy"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "zy", label_positions_spin["zy"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "xz", label_positions_spin["xz"], font_I)

   

    # === 横結合 ===
    max_h_top = max(isop_frames[0].shape[0], isop_frames[1].shape[0])
    max_h_bottom = max(spin_fluo_frame.shape[0], spin_track_frame.shape[0])
    gap_h_bar = np.zeros((max_h_top, gap_h, 3), dtype=np.uint8)
    top = np.hstack([cv2.copyMakeBorder(f, 0, max_h_top - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in isop_frames])
    top = np.hstack((top[:, :base_width], gap_h_bar, top[:, base_width:]))

    gap_h_bar2 = np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8)
    bottom = np.hstack([cv2.copyMakeBorder(f, 0, max_h_bottom - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in [spin_fluo_frame, spin_track_frame]])
    bottom = np.hstack((bottom[:, :base_width], gap_h_bar2, bottom[:, base_width:]))

    gap_v_bar = np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8)
    combined_core = np.vstack((top, gap_v_bar, bottom))

    combined = cv2.copyMakeBorder(combined_core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # 下線の補正
    cv2.rectangle(combined, (side_margin, 400 + top_margin), (503 + side_margin, 402 + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 400 + top_margin), (1104 + gap_h + side_margin, 402 + top_margin), line_color , thickness)
    cv2.rectangle(combined, (side_margin, 1001 + gap_v + top_margin), (513 + side_margin, 1003 + gap_v + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 1001 + gap_v + top_margin), (1114 + gap_h + side_margin, 1003 + gap_v + top_margin), line_color , thickness)    

    # スケールバー
    scale_bar_length = (48, int(44 * resize_ratio))  # isop_fluoのスケールバー長さ
    scale_bar_height = 3
    scale_bar_color = (255, 255, 255)
    scale_bar_position = (437, 280)

    for i in range(2):
        x = gap_h + scale_bar_position[0]
        y = top_margin + (gap_v + base_height + 200) * i + scale_bar_position[1]
        cv2.rectangle(combined, (x, y), (x + scale_bar_length[i], y + scale_bar_height), scale_bar_color, -1)
    

    # 初回に VideoWriter 初期化
    if out is None:
        h, w = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (w, h))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
out.release()

print(f"✅ 出力完了: {output_path}")

✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
Base width: 601, Resize ratio: 0.9966832504145937
処理中: フレーム 0/10 (t = 0.000 s)
✅ 出力完了: combined_output_newtimestamp.mp4


In [88]:
#50Hz確定版
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image


# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)


def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 横幅を合わせるリサイズ ===
def resize_and_add_crosshair_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(h * scale)
    resized = cv2.resize(image, (target_width, new_h))
    return resized


def put_text_pil(img, text, position, font, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)


# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"  # ← 追加：CSVのパス

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")

timestamps = df['Timestamp_T=0'].values  # NumPy配列に変換

# チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ 開けません: {path}")
        else:
            print(f"✅ 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === サイズ・出力 ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio}")

# === FPS・フレーム数取得 === 
# コメントアウトでFPSを変更;; 50.948:終了時刻から算出　50.943:各フレームの時間差平均から算出
fps_isop = 50.948  
#fps_isop = 50.943
n_frames_isop = 9996
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60
base_height = sizes["isop_fluo"][1] # base_heightをisop_fluoの高さに設定

# === ラベル位置の定義をここに追加 ===
label_positions_isop = {
    "xy": (side_margin-10, top_margin+205),
    "zy": (side_margin+540, top_margin + 205),
    "xz": (side_margin-10, top_margin+240)
}
label_positions_spin = {
    "xy": (side_margin-10, top_margin +415),
    "zy": (side_margin + 540, top_margin +415),
    "xz": (side_margin-10, top_margin+ 450)
}


out = None  # 動的作成

# === 時間補正とキャッシュ ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset

last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレームを読み込み、またはダミーフレームを作成
# これにより、最初のフレームが黒くなるのを防ぎます
ret, initial_isop_fluo_frame = caps["isop_fluo"].read()
if ret:
    last_isop_fluo_frame = resize_and_add_crosshair_fixed_width(initial_isop_fluo_frame, base_width)
else:
    last_isop_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8) # 適切なサイズで初期化

ret, initial_isop_track_frame = caps["isop_track"].read()
if ret:
    last_isop_track_frame = resize_and_add_crosshair_fixed_width(initial_isop_track_frame, base_width)
else:
    last_isop_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_fluo_frame = caps["spin_fluo"].read()
if ret:
    last_spin_fluo_frame = resize_and_add_crosshair_fixed_width(initial_spin_fluo_frame, base_width)
else:
    last_spin_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_track_frame = caps["spin_track"].read()
if ret:
    last_spin_track_frame = resize_and_add_crosshair_fixed_width(initial_spin_track_frame, base_width)
else:
    last_spin_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)


for i in range(n_frames_isop): # 処理フレーム数を必要に応じて調整
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames ===
    isop_frames = []
    current_isop_fluo_frame = last_isop_fluo_frame # デフォルトを前のフレームに設定
    current_isop_track_frame = last_isop_track_frame

    # isop_fluo
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        current_isop_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_fluo_frame = current_isop_fluo_frame # 最新のフレームを更新
    isop_frames.append(current_isop_fluo_frame)

    # isop_track
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        current_isop_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_track_frame = current_isop_track_frame # 最新のフレームを更新
    isop_frames.append(current_isop_track_frame)

    # === SPIN fluo frame ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)
    current_spin_fluo_frame = last_spin_fluo_frame # デフォルトを前のフレームに設定

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            current_spin_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
            last_spin_fluo_frame = current_spin_fluo_frame # 最新のフレームを更新
        last_spin_fluo_index = spin_fluo_index
    spin_fluo_frame = current_spin_fluo_frame

    # === SPIN track frame ===
    current_spin_track_frame = last_spin_track_frame # デフォルトを前のフレームに設定
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                current_spin_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
                last_spin_track_frame = current_spin_track_frame # 最新のフレームを更新
            last_spin_track_index = spin_track_index
        spin_track_frame = current_spin_track_frame
    else:
        # 終了時間後に新しいフレームがない場合は、最後の有効なフレームを使用し続ける
        spin_track_frame = last_spin_track_frame

      # クロスライン・外枠
    line_color = (255, 255, 255)
    thickness = -1
    for frame in [isop_frames[0], isop_frames[1], spin_fluo_frame, spin_track_frame]:
        if frame is isop_frames[0] or frame is isop_frames[1]:
            cv2.rectangle(frame, (501, 0), (503, 401), line_color, thickness)
            cv2.rectangle(frame, (0, 301), (601, 303), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (600, 302), line_color, 2)
            cv2.rectangle(frame, (0, 0), (502, 402), line_color, 2)
        else:
            h, w = frame.shape[:2]  # このフレームのサイズに合わせる
            cv2.rectangle(frame, (513, 0), (513, 604), line_color, thickness)
            cv2.rectangle(frame, (0, 513), (604, 513), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (604, 513), line_color, 2)
            cv2.rectangle(frame, (0, 0), (513, 604), line_color, 2)
            cv2.line(frame, (w - 1, 0), (w - 1, 513), line_color, 2)  # 右枠線 ← 修正点

            
    # ラベル・タイマー
    isop_frames[0] = put_text_arial(isop_frames[0], "ISOP microscopy", (5, 5))
    spin_fluo_frame = put_text_arial(spin_fluo_frame, "Spinning disk confocal microscopy", (5, 5))
    isop_frames[0] = put_text_arial(isop_frames[0], f"T = {timestamps[i]:.3f} s", (5, 35))
    
    # xyzラベル
    isop_frames[0] = put_text_pil(isop_frames[0], "xy", label_positions_isop["xy"], font_I)
    isop_frames[0] = put_text_pil(isop_frames[0], "zy", label_positions_isop["zy"], font_I)
    isop_frames[0] = put_text_pil(isop_frames[0], "xz", label_positions_isop["xz"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "xy", label_positions_spin["xy"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "zy", label_positions_spin["zy"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "xz", label_positions_spin["xz"], font_I)

    #スケールバー
    scale_bar_height = 3
    scale_bar_color = (255, 255, 255)

    # ISOPスケールバー
    #isop_pixel_size_um = 0.4095
    #isop_scale_bar_um = 20
    #isop_scale_bar_length = int(round(isop_scale_bar_um / isop_pixel_size_um))  # ≈ 49
    
    isop_bar_x = 430
    isop_bar_y = isop_frames[0].shape[0] - 120
    cv2.rectangle(
        isop_frames[0],
        (isop_bar_x, isop_bar_y),
        (isop_bar_x + 49, isop_bar_y + scale_bar_height),
        scale_bar_color,
        -1
    )

    # SPINスケールバー
    #physical_length_um = 20        # μm
    #pixel_size_um = 0.43           # μm/pixel
   #scale_bar_length_px = int(round(physical_length_um / pixel_size_um))  # 約46.5 → 47
    
    spin_bar_x = 440
    spin_bar_y = spin_fluo_frame.shape[0] - 110
    cv2.rectangle(
        spin_fluo_frame,
        (spin_bar_x, spin_bar_y),
        (spin_bar_x + 47, spin_bar_y + scale_bar_height),
        scale_bar_color,
        -1
    )

    

    # === 横結合 ===
    max_h_top = max(isop_frames[0].shape[0], isop_frames[1].shape[0])
    max_h_bottom = max(spin_fluo_frame.shape[0], spin_track_frame.shape[0])
    gap_h_bar = np.zeros((max_h_top, gap_h, 3), dtype=np.uint8)
    top = np.hstack([cv2.copyMakeBorder(f, 0, max_h_top - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in isop_frames])
    top = np.hstack((top[:, :base_width], gap_h_bar, top[:, base_width:]))

    gap_h_bar2 = np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8)
    bottom = np.hstack([cv2.copyMakeBorder(f, 0, max_h_bottom - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in [spin_fluo_frame, spin_track_frame]])
    bottom = np.hstack((bottom[:, :base_width], gap_h_bar2, bottom[:, base_width:]))

    gap_v_bar = np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8)
    combined_core = np.vstack((top, gap_v_bar, bottom))

    
    combined = cv2.copyMakeBorder(combined_core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # 下線の補正
    cv2.rectangle(combined, (side_margin, 400 + top_margin), (503 + side_margin, 402 + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 400 + top_margin), (1104 + gap_h + side_margin, 402 + top_margin), line_color , thickness)
    cv2.rectangle(combined, (side_margin, 1001 + gap_v + top_margin), (513 + side_margin, 1003 + gap_v + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 1001 + gap_v + top_margin), (1114 + gap_h + side_margin, 1003 + gap_v + top_margin), line_color , thickness)    

  

    # 初回に VideoWriter 初期化
    if out is None:
        h, w = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (w, h))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
out.release()

print(f"✅ 出力完了: {output_path}")

✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/worm1_tdTomato/C1-1_bleachcorrected_full_top90_mean_full_adjustedM957_tdTomato.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi
✅ 開けました: /Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi
Base width: 601, Resize ratio: 0.9966832504145937
処理中: フレーム 0/9996 (t = 0.000 s)
処理中: フレーム 50/9996 (t = 0.981 s)
処理中: フレーム 100/9996 (t = 1.963 s)
処理中: フレーム 150/9996 (t = 2.944 s)
処理中: フレーム 200/9996 (t = 3.926 s)
処理中: フレーム 250/9996 (t = 4.907 s)
処理中: フレーム 300/9996 (t = 5.888 s)
処理中: フレーム 350/9996 (t = 6.870 s)
処理中: フレーム 400/9996 (t = 7.851 s)
処理中: フレーム 450/9996 (t = 8.833 s)
処理中: フレーム 500/9996 (t = 9.814 s)
処理中: フレーム 550/9996 (t = 10.795 s)
処理中: フレーム 600/9996 (t = 11.777 s)
処理中: フレーム 650/9996 (t = 12.758 s)
処理中: フレーム 700/9996 (t = 13.739 s)
処理中: フレーム 750/9996 (t = 14.721 s)
処理中: フレーム 800/9996 (t = 15.702 s)
処理中: フレーム 850/9996 (t = 16

In [ ]:
#10Hzバージョン
import cv2
import numpy as np
import os
import pandas as pd
from PIL import ImageFont, ImageDraw, Image


# === フォント設定 ===
font_path = "/System/Library/Fonts/Supplemental/Arial.ttf"
font_size = 25
font = ImageFont.truetype(font_path, font_size)

font_path_I = "/System/Library/Fonts/Supplemental/Arial Italic.ttf"
font_size_I = 30
font_I = ImageFont.truetype(font_path_I, font_size_I)


def put_text_arial(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def put_text_arial_italic(img, text, position, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font_I, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

# === 横幅を合わせるリサイズ ===
def resize_and_add_crosshair_fixed_width(image, target_width):
    h, w = image.shape[:2]
    scale = target_width / w
    new_h = int(h * scale)
    resized = cv2.resize(image, (target_width, new_h))
    return resized


def put_text_pil(img, text, position, font, color=(255, 255, 255)):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)


# === 入力動画パス ===
paths = {
    "isop_fluo": "/Users/yuusuke/Downloads/Videos/forVideo1-2/1_bleachcorrected_full_top90_mean_full_adjustedG297M1484.avi",
    "isop_track": "/Users/yuusuke/Downloads/Videos/forVideo5/labels_ortho_noscale_comp.avi",
    "spin_fluo": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_fluo_latest.avi",
    "spin_track": "/Users/yuusuke/Downloads/Videos/forVideo5/spin_label_latest.avi"
}
timestamp_csv_path = "Timestamps_worm1.csv"  # ← 追加：CSVのパス

caps = {k: cv2.VideoCapture(paths[k]) for k in paths}

# === タイムスタンプCSV読み込み ===
df = pd.read_csv(timestamp_csv_path)
if 'Timestamp_T=0' not in df.columns:
    raise ValueError("❌ CSVに 'Timestamp_T=0' 列が見つかりません。")

timestamps = df['Timestamp_T=0'].values  # NumPy配列に変換

# チェック
for key, path in paths.items():
    if not os.path.exists(path):
        print(f"❌ ファイルが存在しません: {path}")
    else:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            print(f"❌ 開けません: {path}")
        else:
            print(f"✅ 開けました: {path}")
        cap.release()

if not all(cap.isOpened() for cap in caps.values()):
    raise RuntimeError("❌ 一部の動画が開けません")

# === サイズ・出力 ===
def get_size(cap):
    return int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

sizes = {k: get_size(caps[k]) for k in caps}
base_width = min(s[0] for s in sizes.values())
resize_ratio = base_width / max(s[0] for s in sizes.values())
print(f"Base width: {base_width}, Resize ratio: {resize_ratio}")

# === FPS・フレーム数取得 === 
# コメントアウトでFPSを変更;; 50.948:終了時刻から算出　50.943:各フレームの時間差平均から算出
fps_isop = 50.948  
#fps_isop = 50.943
n_frames_isop = 9996
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = "combined_output_newtimestamp.mp4"

# === マージン・間隔 ===
gap_h = 15
gap_v = 40
side_margin = 20
top_margin = 60
bottom_margin = 60
base_height = sizes["isop_fluo"][1] # base_heightをisop_fluoの高さに設定

# === ラベル位置の定義をここに追加 ===
label_positions_isop = {
    "xy": (side_margin-10, top_margin+205),
    "zy": (side_margin+540, top_margin + 205),
    "xz": (side_margin-10, top_margin+240)
}
label_positions_spin = {
    "xy": (side_margin-10, top_margin +415),
    "zy": (side_margin + 540, top_margin +415),
    "xz": (side_margin-10, top_margin+ 450)
}


out = None  # 動的作成

# === 時間補正とキャッシュ ===
spin_start_offset = 0.31
spin_fps = 3.175
spin_track_max_frames = 146
spin_track_end_time = (spin_track_max_frames - 1) / spin_fps + spin_start_offset

last_spin_fluo_index = -1
last_spin_track_index = -1

# 初期フレームを読み込み、またはダミーフレームを作成
# これにより、最初のフレームが黒くなるのを防ぎます
ret, initial_isop_fluo_frame = caps["isop_fluo"].read()
if ret:
    last_isop_fluo_frame = resize_and_add_crosshair_fixed_width(initial_isop_fluo_frame, base_width)
else:
    last_isop_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8) # 適切なサイズで初期化

ret, initial_isop_track_frame = caps["isop_track"].read()
if ret:
    last_isop_track_frame = resize_and_add_crosshair_fixed_width(initial_isop_track_frame, base_width)
else:
    last_isop_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_fluo_frame = caps["spin_fluo"].read()
if ret:
    last_spin_fluo_frame = resize_and_add_crosshair_fixed_width(initial_spin_fluo_frame, base_width)
else:
    last_spin_fluo_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)

ret, initial_spin_track_frame = caps["spin_track"].read()
if ret:
    last_spin_track_frame = resize_and_add_crosshair_fixed_width(initial_spin_track_frame, base_width)
else:
    last_spin_track_frame = np.zeros((base_height, base_width, 3), dtype=np.uint8)


for i in range(n_frames_isop): # 処理フレーム数を必要に応じて調整
    t = i / fps_isop
    if i % 50 == 0:
        print(f"処理中: フレーム {i}/{n_frames_isop} (t = {t:.3f} s)")

    # === ISOP frames ===
    isop_frames = []
    current_isop_fluo_frame = last_isop_fluo_frame # デフォルトを前のフレームに設定
    current_isop_track_frame = last_isop_track_frame

    # isop_fluo
    caps["isop_fluo"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_fluo"].read()
    if ret:
        current_isop_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_fluo_frame = current_isop_fluo_frame # 最新のフレームを更新
    isop_frames.append(current_isop_fluo_frame)

    # isop_track
    caps["isop_track"].set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = caps["isop_track"].read()
    if ret:
        current_isop_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
        last_isop_track_frame = current_isop_track_frame # 最新のフレームを更新
    isop_frames.append(current_isop_track_frame)

    # === SPIN fluo frame ===
    spin_fluo_index = int(round((t - spin_start_offset + 0.157) * spin_fps))
    spin_fluo_index = max(spin_fluo_index, 0)
    current_spin_fluo_frame = last_spin_fluo_frame # デフォルトを前のフレームに設定

    if spin_fluo_index != last_spin_fluo_index and 0 <= spin_fluo_index < 1000:
        caps["spin_fluo"].set(cv2.CAP_PROP_POS_FRAMES, spin_fluo_index)
        ret, frame = caps["spin_fluo"].read()
        if ret:
            current_spin_fluo_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
            last_spin_fluo_frame = current_spin_fluo_frame # 最新のフレームを更新
        last_spin_fluo_index = spin_fluo_index
    spin_fluo_frame = current_spin_fluo_frame

    # === SPIN track frame ===
    current_spin_track_frame = last_spin_track_frame # デフォルトを前のフレームに設定
    if t <= spin_track_end_time:
        spin_track_index = spin_fluo_index
        if spin_track_index != last_spin_track_index and 0 <= spin_track_index < spin_track_max_frames:
            caps["spin_track"].set(cv2.CAP_PROP_POS_FRAMES, spin_track_index)
            ret, frame = caps["spin_track"].read()
            if ret:
                current_spin_track_frame = resize_and_add_crosshair_fixed_width(frame, base_width)
                last_spin_track_frame = current_spin_track_frame # 最新のフレームを更新
            last_spin_track_index = spin_track_index
        spin_track_frame = current_spin_track_frame
    else:
        # 終了時間後に新しいフレームがない場合は、最後の有効なフレームを使用し続ける
        spin_track_frame = last_spin_track_frame

      # クロスライン・外枠
    line_color = (255, 255, 255)
    thickness = -1
    for frame in [isop_frames[0], isop_frames[1], spin_fluo_frame, spin_track_frame]:
        if frame is isop_frames[0] or frame is isop_frames[1]:
            cv2.rectangle(frame, (501, 0), (503, 401), line_color, thickness)
            cv2.rectangle(frame, (0, 301), (601, 303), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (600, 302), line_color, 2)
            cv2.rectangle(frame, (0, 0), (502, 402), line_color, 2)
        else:
            h, w = frame.shape[:2]  # このフレームのサイズに合わせる
            cv2.rectangle(frame, (513, 0), (513, 604), line_color, thickness)
            cv2.rectangle(frame, (0, 513), (604, 513), line_color, thickness)
            cv2.rectangle(frame, (0, 0), (604, 513), line_color, 2)
            cv2.rectangle(frame, (0, 0), (513, 604), line_color, 2)
            cv2.line(frame, (w - 1, 0), (w - 1, 513), line_color, 2)  # 右枠線 ← 修正点

            
    # ラベル・タイマー
    isop_frames[0] = put_text_arial(isop_frames[0], "ISOP microscopy", (5, 5))
    spin_fluo_frame = put_text_arial(spin_fluo_frame, "Spinning disk confocal microscopy", (5, 5))
    isop_frames[0] = put_text_arial(isop_frames[0], f"T = {timestamps[i]:.3f} s", (5, 35))
    
    # xyzラベル
    isop_frames[0] = put_text_pil(isop_frames[0], "xy", label_positions_isop["xy"], font_I)
    isop_frames[0] = put_text_pil(isop_frames[0], "zy", label_positions_isop["zy"], font_I)
    isop_frames[0] = put_text_pil(isop_frames[0], "xz", label_positions_isop["xz"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "xy", label_positions_spin["xy"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "zy", label_positions_spin["zy"], font_I)
    spin_fluo_frame = put_text_pil(spin_fluo_frame, "xz", label_positions_spin["xz"], font_I)

    #スケールバー
    scale_bar_height = 3
    scale_bar_color = (255, 255, 255)

    # ISOPスケールバー
    #isop_pixel_size_um = 0.4095
    #isop_scale_bar_um = 20
    #isop_scale_bar_length = int(round(isop_scale_bar_um / isop_pixel_size_um))  # ≈ 49
    
    isop_bar_x = 430
    isop_bar_y = isop_frames[0].shape[0] - 120
    cv2.rectangle(
        isop_frames[0],
        (isop_bar_x, isop_bar_y),
        (isop_bar_x + 49, isop_bar_y + scale_bar_height),
        scale_bar_color,
        -1
    )

    # SPINスケールバー
    #physical_length_um = 20        # μm
    #pixel_size_um = 0.43           # μm/pixel
   #scale_bar_length_px = int(round(physical_length_um / pixel_size_um))  # 約46.5 → 47
    
    spin_bar_x = 440
    spin_bar_y = spin_fluo_frame.shape[0] - 110
    cv2.rectangle(
        spin_fluo_frame,
        (spin_bar_x, spin_bar_y),
        (spin_bar_x + 47, spin_bar_y + scale_bar_height),
        scale_bar_color,
        -1
    )

    

    # === 横結合 ===
    max_h_top = max(isop_frames[0].shape[0], isop_frames[1].shape[0])
    max_h_bottom = max(spin_fluo_frame.shape[0], spin_track_frame.shape[0])
    gap_h_bar = np.zeros((max_h_top, gap_h, 3), dtype=np.uint8)
    top = np.hstack([cv2.copyMakeBorder(f, 0, max_h_top - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in isop_frames])
    top = np.hstack((top[:, :base_width], gap_h_bar, top[:, base_width:]))

    gap_h_bar2 = np.zeros((max_h_bottom, gap_h, 3), dtype=np.uint8)
    bottom = np.hstack([cv2.copyMakeBorder(f, 0, max_h_bottom - f.shape[0], 0, 0, cv2.BORDER_CONSTANT) for f in [spin_fluo_frame, spin_track_frame]])
    bottom = np.hstack((bottom[:, :base_width], gap_h_bar2, bottom[:, base_width:]))

    gap_v_bar = np.zeros((gap_v, top.shape[1], 3), dtype=np.uint8)
    combined_core = np.vstack((top, gap_v_bar, bottom))

    
    combined = cv2.copyMakeBorder(combined_core, top_margin, bottom_margin, side_margin, side_margin, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    # 下線の補正
    cv2.rectangle(combined, (side_margin, 400 + top_margin), (503 + side_margin, 402 + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 400 + top_margin), (1104 + gap_h + side_margin, 402 + top_margin), line_color , thickness)
    cv2.rectangle(combined, (side_margin, 1001 + gap_v + top_margin), (513 + side_margin, 1003 + gap_v + top_margin), line_color, thickness)
    cv2.rectangle(combined, (601 + gap_h + side_margin, 1001 + gap_v + top_margin), (1114 + gap_h + side_margin, 1003 + gap_v + top_margin), line_color , thickness)    

  

    # 初回に VideoWriter 初期化
    if out is None:
        h, w = combined.shape[:2]
        out = cv2.VideoWriter(output_path, fourcc, fps_isop, (w, h))

    out.write(combined)

# 終了処理
for cap in caps.values():
    cap.release()
out.release()

print(f"✅ 出力完了: {output_path}")